In [1]:
import pandas as pd
import numpy as np
import pytz
import datetime
from sklearn.metrics import pairwise_distances
from itertools import chain
import math
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import itertools

In [2]:
import sys
display(sys.path)
sys.executable

['/home/soumya/project/project',
 '/home/soumya/Downloads/octeract-engine/octeract/lib',
 '/home/soumya/project/my_project_env/lib/python36.zip',
 '/home/soumya/project/my_project_env/lib/python3.6',
 '/home/soumya/project/my_project_env/lib/python3.6/lib-dynload',
 '/usr/lib/python3.6',
 '',
 '/home/soumya/project/my_project_env/lib/python3.6/site-packages',
 '/home/soumya/project/my_project_env/lib/python3.6/site-packages/pip-20.1b1-py3.6.egg',
 '/home/soumya/project/my_project_env/lib/python3.6/site-packages/IPython/extensions',
 '/home/soumya/.ipython']

'/home/soumya/project/my_project_env/bin/python3'

In [ ]:
del_user = pd.read_csv('userVisits-Delh-allPOI.csv',delimiter=";")
del_user.to_csv("POI-Delhi-dateTaken.csv")
del_poi = pd.read_csv('POI-Delh.csv',delimiter=";")

In [ ]:
del_user

In [ ]:
# Converting given dateTaken to local date and time


Del_str = "POI-Delhi-dateTaken.csv"
def LocalTimeDateTime(Del_str):
    """Localtime date time"""
    del_user = pd.read_csv(Del_str,sep=",",index_col=0)

    pytz.country_timezones('in')
    tz = pytz.timezone('Asia/Kolkata')
    del_user["UTCtime"] = pd.to_datetime(del_user['dateTaken'],unit='s')
    del_user.head()
    del_user["Localtime"]=del_user.UTCtime.dt.tz_localize('UTC').dt.tz_convert(tz)
    del_user.head()
    del_user["Localtime"]=del_user["Localtime"].apply(lambda x:x.strftime("%Y-%m-%d %H:%M:%S"))
    del_user["date"] = del_user["Localtime"].apply(lambda x: x.split(" ")[0])
    del_user["time"] = del_user["Localtime"].apply(lambda x: x.split(" ")[1])

    return del_user
LocalTimeDateTime(Del_str)

In [ ]:
# Travel history of tourists [S=(poi,arrival time,dept time)]

DelhiLocal =LocalTimeDateTime(Del_str)
pd.DataFrame(DelhiLocal.groupby(["userID","seqID","poiID"]).agg({"time":["min","max"]}))

In [ ]:
# Calculating duration(1)
MinMax= DelhiLocal.groupby(["userID","seqID","poiID","poiTheme"]).agg({"time":["min","max"]})
MinMax.columns = MinMax.columns.droplevel()
MinMax = MinMax.reset_index()
MinMax["dur"] = (pd.to_datetime(MinMax["max"])-pd.to_datetime(MinMax["min"]))
MinMax["dur"]= MinMax["dur"].apply(lambda x:x.seconds // 60)
display(MinMax)

# Calculating duration(2) in minutes
Dur_min = LocalTimeDateTime(Del_str)
Dur_min["Minutes"] = Dur_min["Localtime"].apply(lambda x: int(x.split(" ")[1].split(":")[0]) * 60 + int(x.split(" ")[1].split(":")[1]))
Dur_min = Dur_min.groupby(['userID','seqID','poiID','poiTheme']).agg({'Minutes':['min','max']})
Dur_min.columns = Dur_min.columns.droplevel()
Dur_min = Dur_min.reset_index()
Dur_min["dur"]= Dur_min["max"]-Dur_min["min"]
display(Dur_min)

# Total visiting time(1) of tourists
Visit_time1=MinMax.groupby(['userID']).agg({'dur':['sum']})
Visit_time1.columns = Visit_time1.columns.droplevel()
Visit_time1 = Visit_time1.reset_index()
display(Visit_time1)

# Total visiting time(2) of tourists
Visit_time2=Dur_min.groupby(['userID']).agg({'dur':['sum']})
Visit_time2.columns = Visit_time2.columns.droplevel()
Visit_time2 = Visit_time2.reset_index()
display(Visit_time2)

In [ ]:
Interest1 = pd.merge(MinMax,Visit_time1,on="userID",how="left",sort="False")
display(Interest1)
Interest2 = pd.merge(Dur_min,Visit_time2,on="userID",how="left",sort="False")
display(Interest2)

In [ ]:
Interest1["divide"]=Interest1["dur"]/Interest1["sum"]
display(Interest1)
Interest2["divide"]=Interest2["dur"]/Interest2["sum"]
display(Interest2)

In [ ]:
# Interest preference vector of tourists

Interest_pref1=Interest1.groupby(["userID","poiTheme"]).agg({"divide":["sum"]})

display(Interest_pref1)
Interest_pref2=Interest2.groupby(["userID","poiTheme"]).agg({"divide":["sum"]})
Interest_pref2.columns = Interest_pref2.columns.droplevel()
Interest_pref2 = Interest_pref2.reset_index()
display(Interest_pref2)

In [ ]:
# POI Popularity

poi_pop=del_user.groupby(["poiID"]).agg({"poiID":['count']})
poi_pop.columns = poi_pop.columns.droplevel()
poi_pop = poi_pop.reset_index()
display(poi_pop)

In [ ]:
# Tourists-Theme Interest Prefence matrix

sim = pd.DataFrame({"tourist":Interest_pref2['userID'],
                       "theme":Interest_pref2['poiTheme'],
                       "Int_values":Interest_pref2['sum']})

simi = sim.pivot_table(index='tourist',columns='theme',values='Int_values').fillna(0)
simi

In [ ]:
del_user.groupby(["poiTheme"]).agg({"poiTheme":['count']})

In [ ]:
# consider travel sequence containing >=3 POI's

def poiTheme(poiIDlist):
    themelist = []

    for poiID in poiIDlist:
        themelist.append(del_poi[del_poi["poiID"] == poiID]["theme"].tolist()[0])
    return themelist

seqpoicnt = pd.DataFrame(DelhiLocal.groupby(["userID","seqID"])["poiID"].apply(list))
seqpoicnt = seqpoicnt.join(DelhiLocal.groupby(["userID","seqID"])["time"].apply(list))
seqpoicnt["poiTheme"] = seqpoicnt["poiID"].apply(lambda x : poiTheme(x))
seqpoicnt["len"] = seqpoicnt["poiID"].apply(lambda x:len(set(x)))
seqpoicnt = seqpoicnt[seqpoicnt["len"]>=3]
display(seqpoicnt)
seqpoicnt.reset_index()["userID"].value_counts()

In [ ]:
seqpoicnt=seqpoicnt.reset_index()
users=seqpoicnt['userID'].values
display(users)
seq=seqpoicnt['seqID'].values
seq

In [ ]:
modified = pd.DataFrame(columns=['photoID', 'userID', 'dateTaken','poiID','poiTheme','poiFreq','seqID'])

for u in users:
    modified=modified.append(del_user.loc[del_user['userID']== u])
    
modified

In [ ]:
df = pd.DataFrame(columns=['photoID', 'userID', 'dateTaken','poiID','poiTheme','poiFreq','seqID'])

for u in users:
            df=df.append(modified.loc[(modified['userID']==u) & modified['seqID'].isin(seq)])

df

In [ ]:
df=df.reset_index()
df

In [ ]:
# Pre-processed dataframe for k-means clustering

df.drop(columns=['index'])

In [ ]:
simi.reset_index(inplace=True)
simi

In [ ]:
# interest preference of users having >=3 pois

interest = pd.DataFrame(columns=['tourist','Architectural','Building','Museum','Park','Precinct','Religious'])

for u in users:
    interest = interest.append(simi.loc[simi['tourist']== u])
    
interest
interest.drop_duplicates(subset ="tourist", 
                     keep = False, inplace = True)
interest
interest.reset_index(inplace=True)
interest

In [ ]:
# fetch all features into different lists
dff=interest

f1 = dff['tourist'].values
f2 = dff['Architectural'].values
f3 = dff['Building'].values
f4 = dff['Museum'].values
f5 = dff['Park'].values
f6 = dff['Precinct'].values
f7 = dff['Religious'].values

# Storing original data into an array
original_data = np.array(list(zip(f1, f2, f3, f4, f5, f6, f7)))

# data array with only the features which will be used to cluster the data
data = np.array(list(zip(f2, f3, f4, f5, f6, f7)))

data

In [ ]:
groups = [[] for i in range(5)]

def init_centroids(k, data):
    '''
        This function will be used to initialize the centroids once in the beginning.
        Centroids will be randomly chosen as points(features) from the dataset, as this provides faster
        convergence. Another implementation
        can be assigning completely random numbers as centroids, but this is dangerous.
        :param k: (int) number of centroids
        :param data: (np-array) containing the features of the dataset
        :return: (list) 'k' number of randomly selected centroids from the dataset
        '''
    c = []
    # Generate a list of k random numbers between 0 and the # of features of dataset
    s = np.random.randint(low=1, high=len(data), size=k)
    # Check if all members of 's' are unique, if not, generate again
    while (len(s) != len(set(s))):
        s = np.random.randint(low=1, high=len(data), size=k)
    # For every i in s, get ith feature from dataset, and add it to list
    for i in s:
        c.append(data[i])
    return c

def euc_dist(a, b):
    '''
    This function calculates and returns the euclidean distance between two input vectors.
    This is a helper function for cal_dist() to calculate distance of any given point in
    data w.r.t. the centroids.
    :param a: (list) vector a
    :param b: (list) vector b
    :return: (float) euclidean distance between two input vectors
    '''
    sum = 0
    for i, j in zip(a, b):
        a = (i - j) * (i - j)
        sum = sum + a
    return math.sqrt(sum)

def cal_dist(centroids, data):
    '''
    This function will be used to generate the distance table.
    For each point in the given dataset, this function will calculate its euclidean distance
    with respect to each centroid, and a distance table will be generated, which will then be
    used to update the centroid positions.
    :param centroids: (list) containing position of centroids
    :param data: (np-array) containing the features of the dataset
    :return: (list) containing the distances of each point w.r.t. each centroid
    '''
    c_dist = []
    # For each centroid c, iterate through all points in data to calculate its distance from c
    for i in centroids:
        temp = []
        for j in data:
            temp.append(euc_dist(i, j))
        c_dist.append(temp)
    return c_dist

def perf_clustering(k, dist_table):
    '''
    This function will perform clustering on the basis of distance table w.r.t. to all centroids.
    With reference to the distance table, for each point in the table, this function will compare
    its distance from all the centroids, and then cluster the point with the nearest centroid.
    :param k: (int) number of centroids
    :param dist_table: (list) containing the distances of each point w.r.t. each centroid
    :return: (list) containing clusters and indexes of respective members
    '''
    # create empty cluster list of size k
    clusters = []
    for i in range(k):
        clusters.append([])
    # start clustering data points, such that each point is clustered to nearest centroid
    for i in range(len(dist_table[0])):
        d = []
        for j in range(len(dist_table)):
            d.append(dist_table[j][i])
        clusters[d.index(min(d))].append(i)
    return clusters

def update_centroids(centroids, cluster_table, data):
    '''
    This function will update the centroids locations after each iteration
    After performing the clustering, some elements might have migrated from one cluster
    to the another cluster, so w.r.t. the new cluster table, this function will calculate and
    update the new centroid locations
    :param centroids: (list) containing position of centroids
    :param cluster_table: (list) containing clusters and indexes of respective members
    :param data: (np-array) containing the features of the dataset
    :return: (list) containing updated position of centroids
    '''
    for i in range(len(centroids)):
        # Update the centroid if there are some flowers within this centroid
        if (len(cluster_table[i]) > 0):
            temp = []
            # Copy features of cluster members to temp list
            for j in cluster_table[i]:
                temp.append(list(data[j]))
            # Take mean of features of all members of cluster to get new centroid location
            sum = [0] * len(centroids[i])
            for l in temp:
                sum = [(a + b) for a, b in zip(sum, l)]
            centroids[i] = [p / len(temp) for p in sum]

    return centroids


def check_n_stop(dist_mem, cluster_mem):
    '''
    This function will check if the stopping criteria has been met
    This function will help to check if there is a convergence to a solution, with the help of
    specified stopping criterias:
    In case of Iris-Dataset, we can use the following stopping criterias:
    --[1] The euclidean distances/distance table stays unchanged for 2 or more iterations.
    --[2] The cluster table stays unchanged for two or more iterations, i.e. no flower migrations
    :param dist_table: (list) containing the distances of each point w.r.t. each centroid
    :param cluster_table: (list) containing clusters and indexes of respective members
    :return: True, if stopping criteria is met, else False.
    '''

    # Check if distance table has not changed over past iterations
    c1 = all(x == dist_mem[0] for x in dist_mem)
    # Check if cluster table has not changed over past iterations
    c2 = all(y == cluster_mem[0] for y in cluster_mem)

    if c1:
        print("Stopping... Distance table has not changed from few iterations")
    elif c2:
        print("Stopping... Cluster table has not changed from few iterations")
    return c1 or c2

# ================================================================================================
# ==== K-Means ===================================================================================
# ================================================================================================

def kMeans(k, data, max_iterations):
    '''
    Simple implementation of K-Means algorithm
    This function is a very naive implementation of the K-Means algorithm.
    Steps:
    --[1] Initialize random centroids
    --[2] Calculate distances for each point w.r.t. centroid, and store them in distance table.
    --[3] Perform clustering with the available distance table
    --[4] Update the centroid positions
    --[5] Stop if stopping criteria is met, else repeat again from --[2]
    :param k: (int) number of centroids
    :param data: (np-array) containing the features of the dataset
    :param max_iterations: (int) number of maximum iterations allowed.
    :return: none
    '''

    # These lists will maintain memory to check if stopping criteria is met
    dist_mem = []
    cluster_mem = []

    # Initialize centroids
    centroids = init_centroids(k, data)
    # Calculate distance table
    distance_table = cal_dist(centroids, data)
    # Perform clustering based on above generated distance table
    cluster_table = perf_clustering(k, distance_table)
    # Update centroid location based on above generated cluster table
    newCentroids = update_centroids(centroids, cluster_table, data)

    # Add distance and cluster table to memory list
    dist_mem.append(distance_table)
    cluster_mem.append(cluster_table)

    # Repeat from step 2 till stopping criteria is met
    for i in range(max_iterations):
        distance_table = cal_dist(newCentroids, data)
        cluster_table = perf_clustering(k, distance_table)
        newCentroids = update_centroids(newCentroids, cluster_table, data)

        # Check for stopping criteria
        # Maintain memory for past distance table and cluster table
        dist_mem.append(distance_table)
        cluster_mem.append(cluster_table)
        # If distance/cluster has not changed over last 10 iterations, stop, else continue
        if len(dist_mem) > 10:
            dist_mem.pop(0)
            cluster_mem.pop(0)
            if check_n_stop(dist_mem, cluster_mem):
                print("Stopped at iteration #", i)
                break

    # Display the final results
    for i in range(len(newCentroids)):
        print("Centroid #", i, ": ", newCentroids[i])
        print("Members of the cluster: ")
        for j in range(len(cluster_table[i])):
            print(original_data[cluster_table[i][j]])
            groups[i].append(original_data[cluster_table[i][j]][0])

# Run the K-Means algorithm with k = 5, and max-iterations limited to 100

kMeans(5, data, 15)

In [ ]:
# number of tourists in delhi city

len(del_user['userID'].unique())

In [ ]:
groups

In [ ]:
groups[0]

In [ ]:
cos0 = pd.DataFrame(columns=['userID','poiTheme','sum'])

for u in groups[0]:
            cos0=cos0.append(Interest_pref2.loc[(Interest_pref2['userID']==u)])

cos0

In [ ]:
cosi0 = pd.DataFrame({"tourist":cos0['userID'],
                       "theme":cos0['poiTheme'],
                       "Int_values":cos0['sum']})

cosin0 = cosi0.pivot_table(index='tourist',columns='theme',values='Int_values').fillna(0)
cosin0

In [ ]:
similarity0 = 1-pairwise_distances(cosin0, metric="cosine")
cos_sim0=pd.DataFrame(similarity0)
cos_sim0

In [ ]:
summ0 = 0
for i in range(len(groups[0])-1):
    for j in range(i+1,len(groups[0])):
        summ0 = summ0 + cos_sim0[i][j]
        
summ0 = summ0/10
summ0

In [ ]:
cos1 = pd.DataFrame(columns=['userID','poiTheme','sum'])

for u in groups[1]:
            cos1=cos1.append(Interest_pref2.loc[(Interest_pref2['userID']==u)])

cos1

cosi1 = pd.DataFrame({"tourist":cos1['userID'],
                       "theme":cos1['poiTheme'],
                       "Int_values":cos1['sum']})

cosin1 = cosi1.pivot_table(index='tourist',columns='theme',values='Int_values').fillna(0)
cosin1

similarity1 = 1-pairwise_distances(cosin1, metric="cosine")
cos_sim1=pd.DataFrame(similarity1)
cos_sim1

summ1 = 0
for i in range(len(groups[1])-1):
    for j in range(i+1,len(groups[1])):
        summ1 = summ1 + cos_sim1[i][j]
        
summ1 = summ1/len(groups[1])
summ1

In [ ]:
cos2 = pd.DataFrame(columns=['userID','poiTheme','sum'])

for u in groups[2]:
            cos2=cos2.append(Interest_pref2.loc[(Interest_pref2['userID']==u)])

cos2

cosi2 = pd.DataFrame({"tourist":cos2['userID'],
                       "theme":cos2['poiTheme'],
                       "Int_values":cos2['sum']})

cosin2 = cosi2.pivot_table(index='tourist',columns='theme',values='Int_values').fillna(0)
cosin2

similarity2 = 1-pairwise_distances(cosin2, metric="cosine")
cos_sim2=pd.DataFrame(similarity2)
cos_sim2

summ2 = 0
for i in range(len(groups[2])-1):
    for j in range(i+1,len(groups[2])):
        summ2 = summ2 + cos_sim2[i][j]
        
summ2 = summ2/len(groups[2])
summ2

In [ ]:
cos3 = pd.DataFrame(columns=['userID','poiTheme','sum'])

for u in groups[3]:
            cos3=cos3.append(Interest_pref2.loc[(Interest_pref2['userID']==u)])

cos3

cosi3 = pd.DataFrame({"tourist":cos3['userID'],
                       "theme":cos3['poiTheme'],
                       "Int_values":cos3['sum']})

cosin3 = cosi3.pivot_table(index='tourist',columns='theme',values='Int_values').fillna(0)
cosin3

similarity3 = 1-pairwise_distances(cosin3, metric="cosine")
cos_sim3=pd.DataFrame(similarity3)
cos_sim3

summ3 = 0
for i in range(len(groups[3])-1):
    for j in range(i+1,len(groups[3])):
        summ3 = summ3 + cos_sim3[i][j]
        
summ3 = summ3/len(groups[3])
summ3

In [ ]:
cos4 = pd.DataFrame(columns=['userID','poiTheme','sum'])

for u in groups[4]:
            cos4=cos4.append(Interest_pref2.loc[(Interest_pref2['userID']==u)])

cos4

cosi4 = pd.DataFrame({"tourist":cos4['userID'],
                       "theme":cos4['poiTheme'],
                       "Int_values":cos4['sum']})

cosin4 = cosi4.pivot_table(index='tourist',columns='theme',values='Int_values').fillna(0)
cosin4

similarity4 = 1-pairwise_distances(cosin4, metric="cosine")
cos_sim4=pd.DataFrame(similarity4)
cos_sim4

summ4 = 0
for i in range(len(groups[4])-1):
    for j in range(i+1,len(groups[4])):
        summ4 = summ4 + cos_sim4[i][j]
        
summ4 = summ4/len(groups[4])
summ4

In [ ]:
total = (summ0+summ1+summ2+summ3+summ4)/len(groups)
total

# For Toronto City

In [3]:
# Data Pre-processing

tor_user = pd.read_csv('userVisits-Toro.csv',delimiter=";")
tor_user.to_csv("POI-Toro-dateTaken.csv")
tor_poi = pd.read_csv('POI-Toro.csv',delimiter=";")
tor_user

,photoID,userID,dateTaken,poiID,poiTheme,poiFreq,seqID
0,7941504100,10007579@N00,1346844688,30,Structure,1538,1
1,4886005532,10012675@N05,1142731848,6,Cultural,986,2
2,4886006468,10012675@N05,1142732248,6,Cultural,986,2
3,4885404441,10012675@N05,1142732373,6,Cultural,986,2
4,4886008334,10012675@N05,1142732445,6,Cultural,986,2
...,...,...,...,...,...,...,...
39414,8301245106,99906907@N00,1304751632,10,Cultural,346,6055
39415,8300192783,99906907@N00,1304751721,10,Cultural,346,6055
39416,8301245172,99906907@N00,1304757433,10,Cultural,346,6055
39417,8301245324,99906907@N00,1312708790,9,Cultural,278,6056


In [4]:
# Converting given dateTaken to local date and time


tor_str = "POI-Toro-dateTaken.csv"
def LocalTimeDateTime(tor_str):
    """Localtime date time"""
    tor_user = pd.read_csv(tor_str,sep=",",index_col=0)

    pytz.country_timezones('in')
    tz = pytz.timezone('America/Toronto')
    tor_user["UTCtime"] = pd.to_datetime(tor_user['dateTaken'],unit='s')
    tor_user.head()
    tor_user["Localtime"]=tor_user.UTCtime.dt.tz_localize('UTC').dt.tz_convert(tz)
    tor_user.head()
    tor_user["Localtime"]=tor_user["Localtime"].apply(lambda x:x.strftime("%Y-%m-%d %H:%M:%S"))
    tor_user["date"] = tor_user["Localtime"].apply(lambda x: x.split(" ")[0])
    tor_user["time"] = tor_user["Localtime"].apply(lambda x: x.split(" ")[1])

    return tor_user
LocalTimeDateTime(tor_str)

,photoID,userID,dateTaken,poiID,poiTheme,poiFreq,seqID,UTCtime,Localtime,date,time
0,7941504100,10007579@N00,1346844688,30,Structure,1538,1,2012-09-05 11:31:28,2012-09-05 07:31:28,2012-09-05,07:31:28
1,4886005532,10012675@N05,1142731848,6,Cultural,986,2,2006-03-19 01:30:48,2006-03-18 20:30:48,2006-03-18,20:30:48
2,4886006468,10012675@N05,1142732248,6,Cultural,986,2,2006-03-19 01:37:28,2006-03-18 20:37:28,2006-03-18,20:37:28
3,4885404441,10012675@N05,1142732373,6,Cultural,986,2,2006-03-19 01:39:33,2006-03-18 20:39:33,2006-03-18,20:39:33
4,4886008334,10012675@N05,1142732445,6,Cultural,986,2,2006-03-19 01:40:45,2006-03-18 20:40:45,2006-03-18,20:40:45
...,...,...,...,...,...,...,...,...,...,...,...
39414,8301245106,99906907@N00,1304751632,10,Cultural,346,6055,2011-05-07 07:00:32,2011-05-07 03:00:32,2011-05-07,03:00:32
39415,8300192783,99906907@N00,1304751721,10,Cultural,346,6055,2011-05-07 07:02:01,2011-05-07 03:02:01,2011-05-07,03:02:01
39416,8301245172,99906907@N00,1304757433,10,Cultural,346,6055,2011-05-07 08:37:13,2011-05-07 04:37:13,2011-05-07,04:37:13
39417,8301245324,99906907@N00,1312708790,9,Cultural,278,6056,2011-08-07 09:19:50,2011-08-07 05:19:50,2011-08-07,05:19:50


# Definition1: Tourist travel history

In [5]:
# Travel history of tourists [S=(poi,arrival time,dept time)]

TorLocal =LocalTimeDateTime(tor_str)
pd.DataFrame(TorLocal.groupby(["userID","seqID","poiID","poiTheme"]).agg({"time":["min","max"]}))

time          
                                         min       max
userID       seqID poiID poiTheme                     
10007579@N00 1     30    Structure  07:31:28  07:31:28
10012675@N05 2     6     Cultural   20:30:48  20:40:45
             3     6     Cultural   23:48:12  23:48:12
             4     13    Cultural   19:46:14  21:20:48
10014440@N06 5     24    Shopping   20:57:01  21:01:18
...                                      ...       ...
99906907@N00 6053  9     Cultural   04:26:19  05:39:08
             6054  10    Cultural   01:31:54  01:31:54
             6055  10    Cultural   02:43:19  04:37:13
             6056  9     Cultural   05:19:50  05:19:50
             6057  11    Cultural   03:19:44  03:19:44

[7607 rows x 2 columns]

# Total visiting time a tourist t spends at all POIs

In [6]:
# Calculating duration in minutes
tor_Dur_min = LocalTimeDateTime(tor_str)
tor_Dur_min["Minutes"] = tor_Dur_min["Localtime"].apply(lambda x: int(x.split(" ")[1].split(":")[0]) * 60 + int(x.split(" ")[1].split(":")[1]))
tor_Dur_min = tor_Dur_min.groupby(['userID','seqID','poiID','poiTheme']).agg({'Minutes':['min','max']})
tor_Dur_min.columns = tor_Dur_min.columns.droplevel()
tor_Dur_min = tor_Dur_min.reset_index()
tor_Dur_min["dur"]= tor_Dur_min["max"]-tor_Dur_min["min"]
display(tor_Dur_min)

# Total visiting time of tourists
tor_Visit_time2=tor_Dur_min.groupby(['userID']).agg({'dur':['sum']})
tor_Visit_time2.columns = tor_Visit_time2.columns.droplevel()
tor_Visit_time2 = tor_Visit_time2.reset_index()
display(tor_Visit_time2)

,userID,seqID,poiID,poiTheme,min,max,dur
0,10007579@N00,1,30,Structure,451,451,0
1,10012675@N05,2,6,Cultural,1230,1240,10
2,10012675@N05,3,6,Cultural,1428,1428,0
3,10012675@N05,4,13,Cultural,1186,1280,94
4,10014440@N06,5,24,Shopping,1257,1261,4
...,...,...,...,...,...,...,...
7602,99906907@N00,6053,9,Cultural,266,339,73
7603,99906907@N00,6054,10,Cultural,91,91,0
7604,99906907@N00,6055,10,Cultural,163,277,114
7605,99906907@N00,6056,9,Cultural,319,319,0


,userID,sum
0,10007579@N00,0
1,10012675@N05,104
2,10014440@N06,1592
3,10031363@N00,1
4,10116041@N02,0
...,...,...
1390,99436246@N00,40
1391,9975544@N07,0
1392,99771506@N00,554
1393,9985167@N04,1877


In [7]:
tor_Interest2 = pd.merge(tor_Dur_min,tor_Visit_time2,on="userID",how="left",sort="False")
display(tor_Interest2)

tor_Interest2["divide"]=tor_Interest2["dur"]/tor_Interest2["sum"]
display(tor_Interest2)

,userID,seqID,poiID,poiTheme,min,max,dur,sum
0,10007579@N00,1,30,Structure,451,451,0,0
1,10012675@N05,2,6,Cultural,1230,1240,10,104
2,10012675@N05,3,6,Cultural,1428,1428,0,104
3,10012675@N05,4,13,Cultural,1186,1280,94,104
4,10014440@N06,5,24,Shopping,1257,1261,4,1592
...,...,...,...,...,...,...,...,...
7602,99906907@N00,6053,9,Cultural,266,339,73,5256
7603,99906907@N00,6054,10,Cultural,91,91,0,5256
7604,99906907@N00,6055,10,Cultural,163,277,114,5256
7605,99906907@N00,6056,9,Cultural,319,319,0,5256


,userID,seqID,poiID,poiTheme,min,max,dur,sum,divide
0,10007579@N00,1,30,Structure,451,451,0,0,NaN
1,10012675@N05,2,6,Cultural,1230,1240,10,104,0.096154
2,10012675@N05,3,6,Cultural,1428,1428,0,104,0.000000
3,10012675@N05,4,13,Cultural,1186,1280,94,104,0.903846
4,10014440@N06,5,24,Shopping,1257,1261,4,1592,0.002513
...,...,...,...,...,...,...,...,...,...
7602,99906907@N00,6053,9,Cultural,266,339,73,5256,0.013889
7603,99906907@N00,6054,10,Cultural,91,91,0,5256,0.000000
7604,99906907@N00,6055,10,Cultural,163,277,114,5256,0.021689
7605,99906907@N00,6056,9,Cultural,319,319,0,5256,0.000000


# Definition2: Tourist Interest Preference

In [8]:
# Interest preference vector of tourists

tor_Interest_pref2=tor_Interest2.groupby(["userID","poiTheme"]).agg({"divide":["sum"]})
tor_Interest_pref2.columns = tor_Interest_pref2.columns.droplevel()
tor_Interest_pref2 = tor_Interest_pref2.reset_index()
display(tor_Interest_pref2)
tor_Interest_pref2['userID'][0]

,userID,poiTheme,sum
0,10007579@N00,Structure,0.000000
1,10012675@N05,Cultural,1.000000
2,10014440@N06,Amusement,0.854899
3,10014440@N06,Shopping,0.145101
4,10014440@N06,Sport,0.000000
...,...,...,...
3022,9985167@N04,Structure,0.083111
3023,99906907@N00,Beach,0.000000
3024,99906907@N00,Cultural,0.805746
3025,99906907@N00,Shopping,0.194254


'10007579@N00'

# Definition3: POI Popularity

In [9]:
# POI Popularity

tor_poi_pop=tor_user.groupby(["poiID"]).agg({"poiID":['count']})
tor_poi_pop.columns = tor_poi_pop.columns.droplevel()
tor_poi_pop = tor_poi_pop.reset_index()
display(tor_poi_pop)

# Updated poi list

tor_poi_pop0 = pd.DataFrame([[5,0]] , columns=['poiID','count'])
tor_poi_pop = tor_poi_pop.append(tor_poi_pop0,ignore_index=True)
tor_poi_pop = tor_poi_pop.sort_values(by='poiID')
tor_poi_pop = tor_poi_pop.reset_index(drop=True)
display(tor_poi_pop)

,poiID,count
0,1,3506
1,2,609
2,3,688
3,4,3056
4,6,982
5,7,2053
6,8,1736
7,9,278
8,10,346
9,11,4139


,poiID,count
0,1,3506
1,2,609
2,3,688
3,4,3056
4,5,0
5,6,982
6,7,2053
7,8,1736
8,9,278
9,10,346


In [10]:
# Tourists-Theme Interest Prefence matrix

tor_sim = pd.DataFrame({"tourist":tor_Interest_pref2['userID'],
                       "theme":tor_Interest_pref2['poiTheme'],
                       "Int_values":tor_Interest_pref2['sum']})

tor_simi = tor_sim.pivot_table(index='tourist',columns='theme',values='Int_values').fillna(0)
tor_simi

dict_tor_simi = tor_simi.to_dict(orient='index')
dict_tor_simi # Tourist to poi category

{'10007579@N00': {'Amusement': 0.0,
  'Beach': 0.0,
  'Cultural': 0.0,
  'Shopping': 0.0,
  'Sport': 0.0,
  'Structure': 0.0},
 '10012675@N05': {'Amusement': 0.0,
  'Beach': 0.0,
  'Cultural': 1.0,
  'Shopping': 0.0,
  'Sport': 0.0,
  'Structure': 0.0},
 '10014440@N06': {'Amusement': 0.8548994974874372,
  'Beach': 0.0,
  'Cultural': 0.0,
  'Shopping': 0.14510050251256282,
  'Sport': 0.0,
  'Structure': 0.0},
 '10031363@N00': {'Amusement': 0.0,
  'Beach': 0.0,
  'Cultural': 1.0,
  'Shopping': 0.0,
  'Sport': 0.0,
  'Structure': 0.0},
 '10116041@N02': {'Amusement': 0.0,
  'Beach': 0.0,
  'Cultural': 0.0,
  'Shopping': 0.0,
  'Sport': 0.0,
  'Structure': 0.0},
 '10125851@N06': {'Amusement': 0.0,
  'Beach': 0.0,
  'Cultural': 0.0,
  'Shopping': 0.0,
  'Sport': 0.0,
  'Structure': 0.0},
 '101330524@N02': {'Amusement': 0.0,
  'Beach': 0.0,
  'Cultural': 0.0,
  'Shopping': 0.0,
  'Sport': 0.0,
  'Structure': 0.0},
 '101607720@N06': {'Amusement': 0.0,
  'Beach': 0.0,
  'Cultural': 0.0,
  'Shop

In [11]:
# consider travel sequence containing >=3 POI's

def poiTheme(poiIDlist):
    themelist = []

    for poiID in poiIDlist:
        themelist.append(tor_poi[tor_poi["poiID"] == poiID]["theme"].tolist()[0])
    return themelist

tor_seqpoicnt = pd.DataFrame(TorLocal.groupby(["userID","seqID"])["poiID"].apply(list))
tor_seqpoicnt = tor_seqpoicnt.join(TorLocal.groupby(["userID","seqID"])["time"].apply(list))
tor_seqpoicnt["poiTheme"] = tor_seqpoicnt["poiID"].apply(lambda x : poiTheme(x))
tor_seqpoicnt["len"] = tor_seqpoicnt["poiID"].apply(lambda x:len(set(x)))
tor_seqpoicnt = tor_seqpoicnt[tor_seqpoicnt["len"]>=3]
display(tor_seqpoicnt)
tor_seqpoicnt.reset_index()["userID"].value_counts()

poiID  \
userID       seqID                                                      
10502709@N05 58                                 [7, 7, 7, 11, 27, 27]   
             67      [28, 28, 28, 23, 23, 23, 23, 23, 22, 22, 22, 28]   
             71                              [22, 28, 28, 28, 23, 21]   
10627620@N06 99                                       [3, 23, 23, 27]   
             100    [21, 21, 21, 21, 23, 23, 30, 30, 30, 30, 30, 3...   
...                                                               ...   
99436246@N00 5971                    [16, 16, 16, 16, 16, 16, 22, 29]   
99771506@N00 5981   [11, 29, 29, 29, 13, 30, 2, 23, 7, 7, 23, 7, 2...   
9985167@N04  6011                                        [22, 28, 21]   
             6018        [28, 23, 23, 23, 21, 21, 21, 21, 21, 21, 21]   
             6028                         [23, 23, 30, 30, 22, 28, 3]   

                                                                 time  \
userID       seqID                                                      
10502709@N05 58     [20:59:19, 21:52:53, 22:27:41, 01:40:51, 02:04...   
             67     [09:02:44, 10:33:18, 11:03:25, 11:06:52, 11:09...   
             71     [00:18:09, 00:22:42, 00:26:27, 00:28:17, 00:35...   
10627620@N06 99              [22:20:12, 23:44:08, 23:46:17, 00:23:03]   
             100    [18:53:30, 18:53:40, 18:54:56, 18:56:36, 19:42...   
...                                                               ...   
99436246@N00 5971   [02:15:09, 02:32:51, 02:46:25, 02:52:22, 02:52...   
99771506@N00 5981   [02:18:24, 03:13:16, 03:14:09, 03:15:10, 04:33...   
9985167@N04  6011                      [02:44:21, 03:04:24, 04:09:23]   
             6018   [04:25:04, 04:28:09, 04:29:49, 04:58:13, 05:06...   
             6028   [04:42:44, 04:47:14, 07:56:58, 08:02:38, 08:49...   

                                                             poiTheme  len  
userID       seqID                                                          
10502709@N05 58     [Cultural, Cultural, Cultural, Cultural, Shopp...    3  
             67     [Structure, Structure, Structure, Shopping, Sh...    3  
             71     [Beach, Structure, Structure, Structure, Shopp...    4  
10627620@N06 99                 [Sport, Shopping, Shopping, Shopping]    3  
             100    [Beach, Beach, Beach, Beach, Shopping, Shoppin...    3  
...                                                               ...  ...  
99436246@N00 5971   [Amusement, Amusement, Amusement, Amusement, A...    3  
99771506@N00 5981   [Cultural, Structure, Structure, Structure, Cu...    8  
9985167@N04  6011                           [Beach, Structure, Beach]    3  
             6018   [Structure, Shopping, Shopping, Shopping, Beac...    3  
             6028   [Shopping, Shopping, Structure, Structure, Bea...    5  

[335 rows x 4 columns]

20741443@N00    48
84987970@N00    24
34211328@N00    10
37517876@N07     4
39460517@N03     4
                ..
61239510@N00     1
36553196@N08     1
60142746@N00     1
93897327@N00     1
27895744@N00     1
Name: userID, Length: 196, dtype: int64

In [12]:
tor_seqpoicnt=tor_seqpoicnt.reset_index()
tor_users=tor_seqpoicnt['userID'].values
display(tor_users)

array(['10502709@N05', '10502709@N05', '10502709@N05', '10627620@N06',
       '10627620@N06', '11090433@N05', '11191102@N07', '11191102@N07',
       '11191102@N07', '11241023@N03', '11349639@N00', '13273343@N00',
       '13528098@N00', '13644648@N03', '13672753@N00', '13907834@N00',
       '13907834@N00', '14391210@N00', '14391210@N00', '14878709@N00',
       '14878709@N00', '15205252@N00', '16048448@N00', '16326397@N02',
       '16693950@N00', '16693950@N00', '16693950@N00', '16926694@N06',
       '17373191@N00', '18412989@N00', '18412989@N00', '18412989@N00',
       '18948547@N00', '20075574@N00', '20342758@N00', '20344209@N00',
       '20456447@N03', '20456447@N03', '20456447@N03', '20741443@N00',
       '20741443@N00', '20741443@N00', '20741443@N00', '20741443@N00',
       '20741443@N00', '20741443@N00', '20741443@N00', '20741443@N00',
       '20741443@N00', '20741443@N00', '20741443@N00', '20741443@N00',
       '20741443@N00', '20741443@N00', '20741443@N00', '20741443@N00',
      

In [13]:
tor_simi.reset_index(inplace=True)
tor_simi

theme,tourist,Amusement,Beach,Cultural,Shopping,Sport,Structure
0,10007579@N00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,10012675@N05,0.000000,0.000000,1.000000,0.000000,0.0,0.000000
2,10014440@N06,0.854899,0.000000,0.000000,0.145101,0.0,0.000000
3,10031363@N00,0.000000,0.000000,1.000000,0.000000,0.0,0.000000
4,10116041@N02,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...
1390,99436246@N00,1.000000,0.000000,0.000000,0.000000,0.0,0.000000
1391,9975544@N07,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1392,99771506@N00,0.000000,0.075812,0.837545,0.083032,0.0,0.003610
1393,9985167@N04,0.000000,0.848162,0.045818,0.022909,0.0,0.083111


In [14]:
# interest preference of users having >=3 pois

tor_interest = pd.DataFrame(columns=['tourist','Amusement','Beach','Cultural','Shopping','Sport','Structure'])

uniq_tor_users = np.unique(tor_users)
print(uniq_tor_users)

for u in uniq_tor_users:
    tor_interest = tor_interest.append(tor_simi.loc[tor_simi['tourist']== u])
    
tor_interest
tor_interest.drop_duplicates(subset ="tourist", 
                     keep = False, inplace = True)
tor_interest
tor_interest.reset_index(inplace=True)
print(tor_interest)

random_tor_user = tor_interest.sample(n=100)
random_tor_user.reset_index(inplace=True)
random_tor_user

['10502709@N05' '10627620@N06' '11090433@N05' '11191102@N07'
 '11241023@N03' '11349639@N00' '13273343@N00' '13528098@N00'
 '13644648@N03' '13672753@N00' '13907834@N00' '14391210@N00'
 '14878709@N00' '15205252@N00' '16048448@N00' '16326397@N02'
 '16693950@N00' '16926694@N06' '17373191@N00' '18412989@N00'
 '18948547@N00' '20075574@N00' '20342758@N00' '20344209@N00'
 '20456447@N03' '20741443@N00' '22111831@N05' '22979982@N06'
 '22996605@N05' '23119895@N00' '23269969@N00' '23339848@N00'
 '23908938@N02' '23987663@N00' '24128704@N08' '24395354@N02'
 '24616128@N00' '24854893@N00' '25133522@N00' '25475928@N04'
 '25561968@N00' '25802393@N02' '25846522@N00' '26195569@N00'
 '26240579@N05' '26344918@N04' '26736723@N00' '27008470@N00'
 '27031977@N04' '27168489@N00' '27895744@N00' '27947906@N08'
 '28157992@N03' '28288718@N03' '29352917@N00' '29619840@N00'
 '30624156@N00' '30739892@N03' '30855862@N07' '31114347@N05'
 '31349854@N00' '32044903@N04' '32781576@N04' '32827327@N03'
 '32934461@N00' '3294486

theme,level_0,index,tourist,Amusement,Beach,Cultural,Shopping,Sport,Structure
0,183,1335,94725783@N00,0.000000,0.000000,0.384615,0.615385,0.000000,0.000000
1,12,110,14878709@N00,0.399555,0.001483,0.130467,0.208302,0.000000,0.260193
2,116,768,51035555243@N01,0.000000,0.542945,0.000000,0.085890,0.000000,0.371166
3,170,1267,88276719@N00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,28,228,22996605@N05,0.010699,0.000000,0.106356,0.000000,0.882945,0.000000
...,...,...,...,...,...,...,...,...,...
95,123,836,55929357@N00,0.000000,0.441046,0.000000,0.000000,0.000000,0.558954
96,55,382,29619840@N00,0.353659,0.000000,0.402439,0.000000,0.243902,0.000000
97,65,434,32944866@N04,0.000000,0.992537,0.000000,0.000000,0.000000,0.007463
98,177,1312,92487715@N03,0.166979,0.833021,0.000000,0.000000,0.000000,0.000000


# Definition4: Tour Guide Expertise 

In [15]:
# number of visits by an average user

Total_tor_users = len(tor_users)

user_poi = tor_seqpoicnt[['userID','poiID']]
print(user_poi)

dict_user_poi={}

for u in range(Total_tor_users):
    if tor_seqpoicnt['userID'][u] in dict_user_poi:
        dict_user_poi[tor_seqpoicnt['userID'][u]].append(tor_seqpoicnt['poiID'][u])
    else:
        dict_user_poi[tor_seqpoicnt['userID'][u]]=tor_seqpoicnt['poiID'][u]
    
len(dict_user_poi)
print(dict_user_poi)

total_tor_users = len(dict_user_poi)
print ("\n",total_tor_users)
Total = tor_poi_pop['count'].sum()
print (Total,"\n")

print("Denominator:")
denominator=Total/total_tor_users
print(denominator)

           userID                                              poiID
0    10502709@N05                              [7, 7, 7, 11, 27, 27]
1    10502709@N05   [28, 28, 28, 23, 23, 23, 23, 23, 22, 22, 22, 28]
2    10502709@N05                           [22, 28, 28, 28, 23, 21]
3    10627620@N06                                    [3, 23, 23, 27]
4    10627620@N06  [21, 21, 21, 21, 23, 23, 30, 30, 30, 30, 30, 3...
..            ...                                                ...
330  99436246@N00                   [16, 16, 16, 16, 16, 16, 22, 29]
331  99771506@N00  [11, 29, 29, 29, 13, 30, 2, 23, 7, 7, 23, 7, 2...
332   9985167@N04                                       [22, 28, 21]
333   9985167@N04       [28, 23, 23, 23, 21, 21, 21, 21, 21, 21, 21]
334   9985167@N04                        [23, 23, 30, 30, 22, 28, 3]

[335 rows x 2 columns]
{'10502709@N05': [7, 7, 7, 11, 27, 27, [28, 28, 28, 23, 23, 23, 23, 23, 22, 22, 22, 28], [22, 28, 28, 28, 23, 21]], '10627620@N06': [3, 23, 23, 27, 

In [16]:
from collections import Iterable

def flatten(lis):
     for item in lis:
         if isinstance(item, Iterable):
             for x in flatten(item):
                 yield x
         else:        
                 yield item

for u in range(Total_tor_users):
    dict_user_poi[tor_seqpoicnt['userID'][u]] = list(flatten(dict_user_poi[tor_seqpoicnt['userID'][u]]))
    
dict_poi_count = {}
for u in dict_user_poi:
    dict_poi_count[u]={}
    
    
for u in dict_user_poi:   
    for i in range(len(dict_user_poi[u])):
        if dict_user_poi[u][i] in dict_poi_count[u]:
            dict_poi_count[u][dict_user_poi[u][i]] += 1
        else:
            dict_poi_count[u][dict_user_poi[u][i]] = 1
            
            
for u in dict_poi_count:
    for i in dict_poi_count[u]:
        dict_poi_count[u][i]/=(tor_poi_pop['count'][i-1]/total_tor_users)
    
for u in dict_poi_count:
    for i in range(1,31):
        if i not in dict_poi_count[u]:
            dict_poi_count[u][i]=0
            
print(dict_poi_count)

{'10502709@N05': {7: 0.2864101315148563, 11: 0.047354433438028504, 27: 0.6222222222222222, 28: 1.4658119658119657, 23: 0.630225080385852, 22: 0.21759644740494036, 21: 0.05458089668615984, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 8: 0, 9: 0, 10: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 24: 0, 25: 0, 26: 0, 29: 0, 30: 0}, '10627620@N06': {3: 0.28488372093023256, 23: 0.420150053590568, 27: 0.3111111111111111, 21: 0.21832358674463936, 30: 3.1859557867360206, 1: 0, 2: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 22: 0, 24: 0, 25: 0, 26: 0, 28: 0, 29: 0}, '11090433@N05': {3: 1.7093023255813955, 23: 0.52518756698821, 22: 0.4351928948098807, 30: 1.2743823146944082, 1: 0, 2: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0}, '11191102@N07': {16: 0.2758232479594709, 4: 0.06413612565445026, 22:

In [17]:
# Tour Guides to POI

guide_to_poi = pd.DataFrame(dict_poi_count)
guide_to_poi = guide_to_poi.transpose()
guide_to_poi

guide_to_poi = guide_to_poi.reindex(sorted(guide_to_poi.columns), axis=1)
display(guide_to_poi)
guide_to_poi[7]['10502709@N05']

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
10502709@N05,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.286410,0.000000,0.0,0.0,...,0.054581,0.217596,0.630225,0.0,0.000000,0.0,0.622222,1.465812,0.000000,0.000000
10627620@N06,0.000000,0.000000,0.284884,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.218324,0.000000,0.420150,0.0,0.000000,0.0,0.311111,0.000000,0.000000,3.185956
11090433@N05,0.000000,0.000000,1.709302,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.435193,0.525188,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.274382
11191102@N07,0.000000,0.000000,0.000000,0.064136,0.0,0.000000,0.572820,0.000000,0.0,0.0,...,0.000000,0.271996,0.105038,0.0,0.000000,0.0,2.177778,0.000000,0.000000,0.127438
11241023@N03,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,2.959571,0.000000,0.0,0.0,...,0.000000,0.108798,0.000000,0.0,0.000000,0.0,0.000000,1.047009,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9911655@N08,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,4.516129,0.0,0.0,...,0.109162,0.000000,0.000000,0.0,2.074074,0.0,0.000000,1.884615,17.146703,0.127438
99127884@N00,0.167712,0.000000,0.000000,0.000000,0.0,1.796334,0.190940,0.000000,0.0,0.0,...,0.109162,0.435193,0.210075,0.0,0.000000,0.0,0.000000,0.418803,3.429341,0.127438
99436246@N00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.054399,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.263795,0.000000
99771506@N00,0.000000,0.321839,0.000000,0.000000,0.0,0.000000,0.286410,0.000000,0.0,0.0,...,0.054581,0.000000,0.210075,0.0,0.000000,0.0,0.000000,0.000000,0.791386,0.127438


0.2864101315148563

In [18]:
dict_guide_to_poi = {}

list_users = guide_to_poi.index
list_users

for i in list_users:
    for j in range(1,31):
        dict_guide_to_poi[(j,i)] = guide_to_poi[j][i]
        
dict_guide_to_poi    

{(1, '10502709@N05'): 0.0,
 (2, '10502709@N05'): 0.0,
 (3, '10502709@N05'): 0.0,
 (4, '10502709@N05'): 0.0,
 (5, '10502709@N05'): 0.0,
 (6, '10502709@N05'): 0.0,
 (7, '10502709@N05'): 0.2864101315148563,
 (8, '10502709@N05'): 0.0,
 (9, '10502709@N05'): 0.0,
 (10, '10502709@N05'): 0.0,
 (11, '10502709@N05'): 0.047354433438028504,
 (12, '10502709@N05'): 0.0,
 (13, '10502709@N05'): 0.0,
 (14, '10502709@N05'): 0.0,
 (15, '10502709@N05'): 0.0,
 (16, '10502709@N05'): 0.0,
 (17, '10502709@N05'): 0.0,
 (18, '10502709@N05'): 0.0,
 (19, '10502709@N05'): 0.0,
 (20, '10502709@N05'): 0.0,
 (21, '10502709@N05'): 0.05458089668615984,
 (22, '10502709@N05'): 0.21759644740494036,
 (23, '10502709@N05'): 0.630225080385852,
 (24, '10502709@N05'): 0.0,
 (25, '10502709@N05'): 0.0,
 (26, '10502709@N05'): 0.0,
 (27, '10502709@N05'): 0.6222222222222222,
 (28, '10502709@N05'): 1.4658119658119657,
 (29, '10502709@N05'): 0.0,
 (30, '10502709@N05'): 0.0,
 (1, '10627620@N06'): 0.0,
 (2, '10627620@N06'): 0.0,
 (3, '1

# Step1: Tourist to Tour Group Allocation (Tourist2Group)

In [19]:
# fetch all features into different lists
dff1 = random_tor_user

f1 = dff1['tourist'].values
f2 = dff1['Amusement'].values
f3 = dff1['Beach'].values
f4 = dff1['Cultural'].values
f5 = dff1['Shopping'].values
f6 = dff1['Sport'].values
f7 = dff1['Structure'].values

# Storing original data into an array
tor_original_data = np.array(list(zip(f1, f2, f3, f4, f5, f6, f7)))

# data array with only the features which will be used to cluster the data
tor_data = np.array(list(zip(f2, f3, f4, f5, f6, f7)))

display(tor_data)
len(tor_data)

array([[0.00000000e+00, 0.00000000e+00, 3.84615385e-01, 6.15384615e-01,
        0.00000000e+00, 0.00000000e+00],
       [3.99555226e-01, 1.48257969e-03, 1.30467013e-01, 2.08302446e-01,
        0.00000000e+00, 2.60192735e-01],
       [0.00000000e+00, 5.42944785e-01, 0.00000000e+00, 8.58895706e-02,
        0.00000000e+00, 3.71165644e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [1.06985525e-02, 0.00000000e+00, 1.06356199e-01, 0.00000000e+00,
        8.82945249e-01, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [8.16753927e-01, 0.00000000e+00, 1.83246073e-01, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [2.05761317e-02, 0.00000000e+00, 9.76680384e-01, 2.74348422e-03,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 

100

In [20]:
tor_groups = [[] for i in range(5)]

def init_centroids(k, data):
    '''
        This function will be used to initialize the centroids once in the beginning.
        Centroids will be randomly chosen as points(features) from the dataset, as this provides faster
        convergence. Another implementation
        can be assigning completely random numbers as centroids, but this is dangerous.
        :param k: (int) number of centroids
        :param data: (np-array) containing the features of the dataset
        :return: (list) 'k' number of randomly selected centroids from the dataset
        '''
    c = []
    # Generate a list of k random numbers between 0 and the # of features of dataset
    s = np.random.randint(low=1, high=len(data), size=k)
    # Check if all members of 's' are unique, if not, generate again
    while (len(s) != len(set(s))):
        s = np.random.randint(low=1, high=len(data), size=k)
    # For every i in s, get ith feature from dataset, and add it to list
    for i in s:
        c.append(data[i])
    return c

def euc_dist(a, b):
    '''
    This function calculates and returns the euclidean distance between two input vectors.
    This is a helper function for cal_dist() to calculate distance of any given point in
    data w.r.t. the centroids.
    :param a: (list) vector a
    :param b: (list) vector b
    :return: (float) euclidean distance between two input vectors
    '''
    sum = 0
    for i, j in zip(a, b):
        a = (i - j) * (i - j)
        sum = sum + a
    return math.sqrt(sum)

def cal_dist(centroids, data):
    '''
    This function will be used to generate the distance table.
    For each point in the given dataset, this function will calculate its euclidean distance
    with respect to each centroid, and a distance table will be generated, which will then be
    used to update the centroid positions.
    :param centroids: (list) containing position of centroids
    :param data: (np-array) containing the features of the dataset
    :return: (list) containing the distances of each point w.r.t. each centroid
    '''
    c_dist = []
    # For each centroid c, iterate through all points in data to calculate its distance from c
    for i in centroids:
        temp = []
        for j in data:
            temp.append(euc_dist(i, j))
        c_dist.append(temp)
    return c_dist

def perf_clustering(k, dist_table):
    '''
    This function will perform clustering on the basis of distance table w.r.t. to all centroids.
    With reference to the distance table, for each point in the table, this function will compare
    its distance from all the centroids, and then cluster the point with the nearest centroid.
    :param k: (int) number of centroids
    :param dist_table: (list) containing the distances of each point w.r.t. each centroid
    :return: (list) containing clusters and indexes of respective members
    '''
    # create empty cluster list of size k
    clusters = []
    for i in range(k):
        clusters.append([])
    # start clustering data points, such that each point is clustered to nearest centroid
    for i in range(len(dist_table[0])):
        d = []
        for j in range(len(dist_table)):
            d.append(dist_table[j][i])
        clusters[d.index(min(d))].append(i)
    return clusters

def update_centroids(centroids, cluster_table, data):
    '''
    This function will update the centroids locations after each iteration
    After performing the clustering, some elements might have migrated from one cluster
    to the another cluster, so w.r.t. the new cluster table, this function will calculate and
    update the new centroid locations
    :param centroids: (list) containing position of centroids
    :param cluster_table: (list) containing clusters and indexes of respective members
    :param data: (np-array) containing the features of the dataset
    :return: (list) containing updated position of centroids
    '''
    for i in range(len(centroids)):
        # Update the centroid if there are some flowers within this centroid
        if (len(cluster_table[i]) > 0):
            temp = []
            # Copy features of cluster members to temp list
            for j in cluster_table[i]:
                temp.append(list(data[j]))
            # Take mean of features of all members of cluster to get new centroid location
            sum = [0] * len(centroids[i])
            for l in temp:
                sum = [(a + b) for a, b in zip(sum, l)]
            centroids[i] = [p / len(temp) for p in sum]

    return centroids


def check_n_stop(dist_mem, cluster_mem):
    '''
    This function will check if the stopping criteria has been met
    This function will help to check if there is a convergence to a solution, with the help of
    specified stopping criterias:
    In case of Iris-Dataset, we can use the following stopping criterias:
    --[1] The euclidean distances/distance table stays unchanged for 2 or more iterations.
    --[2] The cluster table stays unchanged for two or more iterations, i.e. no flower migrations
    :param dist_table: (list) containing the distances of each point w.r.t. each centroid
    :param cluster_table: (list) containing clusters and indexes of respective members
    :return: True, if stopping criteria is met, else False.
    '''

    # Check if distance table has not changed over past iterations
    c1 = all(x == dist_mem[0] for x in dist_mem)
    # Check if cluster table has not changed over past iterations
    c2 = all(y == cluster_mem[0] for y in cluster_mem)

    if c1:
        print("Stopping... Distance table has not changed from few iterations")
    elif c2:
        print("Stopping... Cluster table has not changed from few iterations")
    return c1 or c2

# ================================================================================================
# ==== K-Means ===================================================================================
# ================================================================================================

def kMeans(k, data, max_iterations):
    '''
    Simple implementation of K-Means algorithm
    This function is a very naive implementation of the K-Means algorithm.
    Steps:
    --[1] Initialize random centroids
    --[2] Calculate distances for each point w.r.t. centroid, and store them in distance table.
    --[3] Perform clustering with the available distance table
    --[4] Update the centroid positions
    --[5] Stop if stopping criteria is met, else repeat again from --[2]
    :param k: (int) number of centroids
    :param data: (np-array) containing the features of the dataset
    :param max_iterations: (int) number of maximum iterations allowed.
    :return: none
    '''

    # These lists will maintain memory to check if stopping criteria is met
    dist_mem = []
    cluster_mem = []

    # Initialize centroids
    centroids = init_centroids(k, data)
    # Calculate distance table
    distance_table = cal_dist(centroids, data)
    # Perform clustering based on above generated distance table
    cluster_table = perf_clustering(k, distance_table)
    # Update centroid location based on above generated cluster table
    newCentroids = update_centroids(centroids, cluster_table, data)

    # Add distance and cluster table to memory list
    dist_mem.append(distance_table)
    cluster_mem.append(cluster_table)

    # Repeat from step 2 till stopping criteria is met
    for i in range(max_iterations):
        distance_table = cal_dist(newCentroids, data)
        cluster_table = perf_clustering(k, distance_table)
        newCentroids = update_centroids(newCentroids, cluster_table, data)

        # Check for stopping criteria
        # Maintain memory for past distance table and cluster table
        dist_mem.append(distance_table)
        cluster_mem.append(cluster_table)
        # If distance/cluster has not changed over last 10 iterations, stop, else continue
        if len(dist_mem) > 10:
            dist_mem.pop(0)
            cluster_mem.pop(0)
            if check_n_stop(dist_mem, cluster_mem):
                print("Stopped at iteration #", i)
                break

    # Display the final results
    for i in range(len(newCentroids)):
        print("Centroid #", i, ": ", newCentroids[i])
        print("Members of the cluster: ")
        for j in range(len(cluster_table[i])):
            print(tor_original_data[cluster_table[i][j]])
            tor_groups[i].append(tor_original_data[cluster_table[i][j]][0])

# Run the K-Means algorithm with k = 5, and max-iterations limited to 100

kMeans(5, tor_data, 100)

Stopping... Cluster table has not changed from few iterations
Stopped at iteration # 12
Centroid # 0 :  [0.09275466420513019, 0.05203644759349384, 0.1727289065163948, 0.0030119572117798133, 0.6788242397547145, 0.0006437847184869251]
Members of the cluster: 
['22996605@N05' '0.010698552548772814' '0.0' '0.10635619886721209' '0.0'
 '0.8829452485840151' '0.0']
['62373848@N00' '0.4719185827365247' '0.0' '0.0' '0.0'
 '0.5280814172634754' '0.0']
['98185064@N00' '0.0' '0.0' '0.0851063829787234' '0.0'
 '0.9148936170212766' '0.0']
['41894165897@N01' '0.0' '0.0' '0.0' '0.0' '0.99830220713073'
 '0.001697792869269949']
['23908938@N02' '0.07964879272499216' '0.0' '0.4700533082470994' '0.0'
 '0.4477892756349953' '0.002508623392913139']
['42907325@N00' '0.0' '0.0' '0.47431223069274114' '0.003977461054027179'
 '0.5217103082532317' '0.0']
['64834051@N00' '0.0' '0.0' '0.13548387096774192' '0.0'
 '0.864516129032258' '0.0']
['35764233@N00' '0.0' '0.0007415647015202076' '0.5239154616240267'
 '0.01075268817

In [21]:
# Tour Groups

tor_groups

[['22996605@N05',
  '62373848@N00',
  '98185064@N00',
  '41894165897@N01',
  '23908938@N02',
  '42907325@N00',
  '64834051@N00',
  '35764233@N00',
  '17373191@N00',
  '47501960@N00',
  '84518681@N00'],
 ['51035555243@N01',
  '61239510@N00',
  '77112637@N00',
  '39587684@N00',
  '30739892@N03',
  '41721440@N00',
  '31349854@N00',
  '24854893@N00',
  '13528098@N00',
  '35034347485@N01',
  '43334562@N00',
  '81471618@N06',
  '8070489@N02',
  '35034348378@N01',
  '91255327@N00',
  '39038071@N00',
  '9911655@N08',
  '11090433@N05',
  '10627620@N06',
  '84035351@N00',
  '34619038@N00',
  '34608255@N08',
  '8250978@N04',
  '9161595@N03',
  '32934461@N00',
  '84987970@N00',
  '55929357@N00',
  '32944866@N04',
  '92487715@N03'],
 ['14878709@N00',
  '88276719@N00',
  '99436246@N00',
  '89613559@N07',
  '57519914@N00',
  '16326397@N02',
  '13273343@N00',
  '26195569@N00',
  '70554294@N00',
  '65438265@N00',
  '27895744@N00',
  '44359705@N07',
  '43919105@N02',
  '93241698@N00',
  '60142746@N00',


In [22]:
cosinT01_df01 = pd.DataFrame(columns=['Amusement','Beach','Cultural','Shopping','Sport','Structure'])
jac_tor_total=0
tor_total=0
sum_com_top_int=0
for k in range(len(tor_groups)):

    cosT0 = pd.DataFrame(columns=['userID','poiTheme','sum'])

    for u in tor_groups[k]:
            cosT0=cosT0.append(tor_Interest_pref2.loc[(tor_Interest_pref2['userID']==u)])

    cosT0

    cosiT0 = pd.DataFrame({"tourist":cosT0['userID'],
                       "theme":cosT0['poiTheme'],
                       "Int_values":cosT0['sum']})

    cosinT0 = cosiT0.pivot_table(index='tourist',columns='theme',values='Int_values').fillna(0)
    display(cosinT0)

    dict_com = {}
    dict_com = cosinT0.to_dict()
    display(dict_com)
    #print(dict_com[0][0])
    
    # for group interest preference
    cosinT01_df = cosinT0.mean(axis = 0, skipna = True)
    cosinT01_df01 = cosinT01_df01.append({'Amusement':cosinT01_df['Amusement'],'Beach':cosinT01_df['Beach'],'Cultural':cosinT01_df['Cultural'],'Shopping':cosinT01_df['Shopping'],'Sport':cosinT01_df['Sport'],'Structure':cosinT01_df['Structure']},ignore_index=True) 
    
    # Jaccard Similarity
    jac_similarityT0 = 1-pairwise_distances(cosinT0, metric="hamming")
    jac_simT0=pd.DataFrame(jac_similarityT0)
    jac_simT0

    jac_summT0 = 0
    jac_count=0
    for i in range(len(tor_groups[k])-1):
        for j in range(i+1,len(tor_groups[k])):
            jac_count+=1
            jac_summT0 = jac_summT0 + jac_simT0[i][j]
        
    jac_summT0 = jac_summT0/jac_count
    jac_summT0
    
    jac_tor_total=jac_tor_total+jac_summT0
    
    # Cosine Similarity
    similarityT0 = 1-pairwise_distances(cosinT0, metric="cosine")
    cos_simT0=pd.DataFrame(similarityT0)
    cos_simT0

    summT0 = 0
    count=0
    for i in range(len(tor_groups[k])-1):
        for j in range(i+1,len(tor_groups[k])):
            count+=1
            summT0 = summT0 + cos_simT0[i][j]
        
    summT0 = summT0/count
    summT0
    
    tor_total=tor_total+summT0
    
    # Common top interest
    cat_list=['Amusement','Beach','Cultural','Shopping','Sport','Structure']
    com_top_int = 0
    com_list = []
    for i in cat_list:
        com_sum = 0
        for j in tor_groups[k]:
            com_sum = com_sum + dict_com[i][j]
        com_sum = com_sum / len(tor_groups[k])
        com_list.append(com_sum)
    com_top_int = max(com_list)
    sum_com_top_int += com_top_int
    
jac_tor_total=jac_tor_total/len(tor_groups)    
tor_total=tor_total/len(tor_groups)
sum_com_top_int=sum_com_top_int/len(tor_groups)
print("--------Result of k-means algorithm--------\n")
print("jaccard similarity: ",jac_tor_total)
print("cosine similarity: ",tor_total)
print("common top interest: ",sum_com_top_int)
print("\n")
print(cosinT01_df01, sep='\n')

theme,Amusement,Beach,Cultural,Shopping,Sport,Structure
tourist,,,,,,
17373191@N00,0.000000,0.446875,0.000000,0.000000,0.553125,0.000000
22996605@N05,0.010699,0.000000,0.106356,0.000000,0.882945,0.000000
23908938@N02,0.079649,0.000000,0.470053,0.000000,0.447789,0.002509
35764233@N00,0.000000,0.000742,0.523915,0.010753,0.464590,0.000000
41894165897@N01,0.000000,0.000000,0.000000,0.000000,0.998302,0.001698
42907325@N00,0.000000,0.000000,0.474312,0.003977,0.521710,0.000000
47501960@N00,0.000000,0.124784,0.060380,0.018401,0.793560,0.002875
62373848@N00,0.471919,0.000000,0.000000,0.000000,0.528081,0.000000
64834051@N00,0.000000,0.000000,0.135484,0.000000,0.864516,0.000000


{'Amusement': {'17373191@N00': 0.0,
  '22996605@N05': 0.010698552548772814,
  '23908938@N02': 0.07964879272499216,
  '35764233@N00': 0.0,
  '41894165897@N01': 0.0,
  '42907325@N00': 0.0,
  '47501960@N00': 0.0,
  '62373848@N00': 0.4719185827365247,
  '64834051@N00': 0.0,
  '84518681@N00': 0.45803537824614227,
  '98185064@N00': 0.0},
 'Beach': {'17373191@N00': 0.44687499999999997,
  '22996605@N05': 0.0,
  '23908938@N02': 0.0,
  '35764233@N00': 0.0007415647015202076,
  '41894165897@N01': 0.0,
  '42907325@N00': 0.0,
  '47501960@N00': 0.12478435882691201,
  '62373848@N00': 0.0,
  '64834051@N00': 0.0,
  '84518681@N00': 0.0,
  '98185064@N00': 0.0},
 'Cultural': {'17373191@N00': 0.0,
  '22996605@N05': 0.10635619886721209,
  '23908938@N02': 0.4700533082470994,
  '35764233@N00': 0.5239154616240267,
  '41894165897@N01': 0.0,
  '42907325@N00': 0.47431223069274114,
  '47501960@N00': 0.060379528464634846,
  '62373848@N00': 0.0,
  '64834051@N00': 0.13548387096774192,
  '84518681@N00': 0.0444109898381

theme,Amusement,Beach,Cultural,Shopping,Sport,Structure
tourist,,,,,,
10627620@N06,0.000000,0.001295,0.009712,0.003885,0.025575,0.959534
11090433@N05,0.000000,0.430607,0.021066,0.001239,0.323420,0.223668
13528098@N00,0.138298,0.000000,0.000000,0.000000,0.000000,0.861702
24854893@N00,0.000000,0.010180,0.030540,0.003915,0.000000,0.955364
30739892@N03,0.012891,0.885206,0.000000,0.085942,0.000000,0.015961
31349854@N00,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
32934461@N00,0.167147,0.582133,0.242075,0.000000,0.000000,0.008646
32944866@N04,0.000000,0.992537,0.000000,0.000000,0.000000,0.007463
34608255@N08,0.000000,0.486486,0.148649,0.000000,0.000000,0.364865


{'Amusement': {'10627620@N06': 0.0,
  '11090433@N05': 0.0,
  '13528098@N00': 0.13829787234042554,
  '24854893@N00': 0.0,
  '30739892@N03': 0.01289134438305709,
  '31349854@N00': 0.0,
  '32934461@N00': 0.16714697406340057,
  '32944866@N04': 0.0,
  '34608255@N08': 0.0,
  '34619038@N00': 0.0,
  '35034347485@N01': 0.0,
  '35034348378@N01': 0.0,
  '39038071@N00': 0.0,
  '39587684@N00': 0.0,
  '41721440@N00': 0.0,
  '43334562@N00': 0.0,
  '51035555243@N01': 0.0,
  '55929357@N00': 0.0,
  '61239510@N00': 0.0,
  '77112637@N00': 0.1725067385444744,
  '8070489@N02': 0.0,
  '81471618@N06': 0.014373716632443531,
  '8250978@N04': 0.0,
  '84035351@N00': 0.0,
  '84987970@N00': 0.0988295777729162,
  '91255327@N00': 0.0,
  '9161595@N03': 0.4098360655737705,
  '92487715@N03': 0.1669793621013133,
  '9911655@N08': 0.0},
 'Beach': {'10627620@N06': 0.0012949174490126255,
  '11090433@N05': 0.4306071871127633,
  '13528098@N00': 0.0,
  '24854893@N00': 0.010180109631949883,
  '30739892@N03': 0.8852056476365869,


theme,Amusement,Beach,Cultural,Shopping,Sport,Structure
tourist,,,,,,
13273343@N00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
14878709@N00,0.399555,0.001483,0.130467,0.208302,0.000000,0.260193
15205252@N00,0.812500,0.000000,0.125000,0.000000,0.062500,0.000000
16326397@N02,0.825157,0.035220,0.130818,0.000000,0.008805,0.000000
26195569@N00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
27168489@N00,0.916667,0.055556,0.027778,0.000000,0.000000,0.000000
27895744@N00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
29619840@N00,0.353659,0.000000,0.402439,0.000000,0.243902,0.000000
43919105@N02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


...

theme,Amusement,Beach,Cultural,Shopping,Sport,Structure
tourist,,,,,,
10502709@N05,0.000000,0.148921,0.279054,0.366040,0.000000,0.205985
20342758@N00,0.462533,0.000000,0.035236,0.502230,0.000000,0.000000
24128704@N08,0.333333,0.000000,0.000000,0.666667,0.000000,0.000000
26240579@N05,0.281250,0.000000,0.000000,0.718750,0.000000,0.000000
29352917@N00,0.114014,0.000000,0.372922,0.149644,0.000000,0.363420
31114347@N05,0.000000,0.000000,0.000000,0.653846,0.000000,0.346154
33547369@N00,0.000000,0.501007,0.000000,0.483210,0.000000,0.015782
39357749@N00,0.037037,0.000000,0.000000,0.296296,0.000000,0.666667
44124367252@N01,0.000000,0.166323,0.000515,0.735839,0.097322,0.000000


{'Amusement': {'10502709@N05': 0.0,
  '20342758@N00': 0.4625334522747547,
  '24128704@N08': 0.3333333333333333,
  '26240579@N05': 0.28125,
  '29352917@N00': 0.11401425178147269,
  '31114347@N05': 0.0,
  '33547369@N00': 0.0,
  '39357749@N00': 0.037037037037037035,
  '44124367252@N01': 0.0,
  '61609579@N00': 0.0,
  '91032493@N00': 0.2246376811594203,
  '93897327@N00': 0.0,
  '9449875@N03': 0.22727853764540457,
  '94725783@N00': 0.0},
 'Beach': {'10502709@N05': 0.1489213639526792,
  '20342758@N00': 0.0,
  '24128704@N08': 0.0,
  '26240579@N05': 0.0,
  '29352917@N00': 0.0,
  '31114347@N05': 0.0,
  '33547369@N00': 0.5010073875083949,
  '39357749@N00': 0.0,
  '44124367252@N01': 0.16632337796086508,
  '61609579@N00': 0.23076923076923078,
  '91032493@N00': 0.06521739130434782,
  '93897327@N00': 0.0,
  '9449875@N03': 0.028377860155950403,
  '94725783@N00': 0.0},
 'Cultural': {'10502709@N05': 0.27905358385525403,
  '20342758@N00': 0.0352363960749331,
  '24128704@N08': 0.0,
  '26240579@N05': 0.0,


theme,Amusement,Beach,Cultural,Shopping,Sport,Structure
tourist,,,,,,
16926694@N06,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
20075574@N00,0.000000,0.350000,0.570000,0.080000,0.000000,0.000000
23339848@N00,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
24395354@N02,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
26344918@N04,0.000000,0.000000,0.983333,0.000000,0.016667,0.000000
27947906@N08,0.205882,0.000000,0.794118,0.000000,0.000000,0.000000
28157992@N03,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
32044903@N04,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
33906853@N00,0.000000,0.034091,0.625000,0.000000,0.340909,0.000000


{'Amusement': {'16926694@N06': 0.0,
  '20075574@N00': 0.0,
  '23339848@N00': 0.0,
  '24395354@N02': 0.0,
  '26344918@N04': 0.0,
  '27947906@N08': 0.20588235294117646,
  '28157992@N03': 0.0,
  '32044903@N04': 0.0,
  '33906853@N00': 0.0,
  '34961066@N00': 0.0,
  '36357702@N08': 0.0,
  '47554402@N00': 0.0,
  '51035681908@N01': 0.0205761316872428,
  '70289230@N00': 0.0,
  '71206023@N00': 0.0,
  '71482738@N00': 0.0007369196757553427,
  '73416633@N00': 0.0,
  '82845843@N04': 0.0,
  '85801042@N00': 0.021059782608695652,
  '96684313@N00': 0.0,
  '99127884@N00': 0.0,
  '99771506@N00': 0.0},
 'Beach': {'16926694@N06': 0.0,
  '20075574@N00': 0.35,
  '23339848@N00': 0.0,
  '24395354@N02': 0.0,
  '26344918@N04': 0.0,
  '27947906@N08': 0.0,
  '28157992@N03': 0.0,
  '32044903@N04': 0.0,
  '33906853@N00': 0.03409090909090909,
  '34961066@N00': 0.0024813895781637717,
  '36357702@N08': 0.05454545454545454,
  '47554402@N00': 0.0,
  '51035681908@N01': 0.0,
  '70289230@N00': 0.0035587188612099642,
  '71206

--------Result of k-means algorithm--------

jaccard similarity:  0.36232004460640127
cosine similarity:  0.6812544298862436
common top interest:  0.6213317851915291


   Amusement     Beach  Cultural  Shopping     Sport  Structure
0   0.092755  0.052036  0.172729  0.003012  0.678824   0.000644
1   0.040719  0.507440  0.031577  0.035251  0.028749   0.356263
2   0.527512  0.020682  0.100179  0.019096  0.015312   0.025551
3   0.120006  0.081473  0.116463  0.525091  0.025759   0.131208
4   0.011284  0.037635  0.867791  0.026152  0.016253   0.040884


In [23]:
cosT01 = pd.DataFrame(columns=['userID','poiTheme','sum'])

for u in tor_groups[0]:
            cosT01=cosT01.append(tor_Interest_pref2.loc[(tor_Interest_pref2['userID']==u)])

cosT01

cosiT01 = pd.DataFrame({"tourist":cosT01['userID'],
                       "theme":cosT01['poiTheme'],
                       "Int_values":cosT01['sum']})

cosinT01 = cosiT01.pivot_table(index='tourist',columns='theme',values='Int_values').fillna(0)
cosinT01

similarityT01 = 1-pairwise_distances(cosinT01, metric="cosine")
cos_simT01=pd.DataFrame(similarityT01)
display(cos_simT01)

summT01 = 0
count = 0
for i in range(len(tor_groups[0])-1):
        for j in range(i+1,len(tor_groups[0])):
            count+=1
            summT01 = summT01 + cos_simT01[i][j]
        
summT01 = summT01/count
summT01

,0,1,2,3,4,5,6,7,8,9,10
0,1.000000,0.772220,0.532532,0.516695,0.777857,0.575543,0.863370,0.580006,0.768479,0.571056,0.774515
1,0.772220,1.000000,0.767054,0.748050,0.992750,0.814985,0.986642,0.748255,0.999295,0.744783,0.999560
2,0.532532,0.767054,1.000000,0.991801,0.684619,0.989977,0.728084,0.591621,0.787625,0.631991,0.748234
3,0.516695,0.748050,0.991801,1.000000,0.663396,0.994181,0.709899,0.494659,0.771225,0.536048,0.729838
4,0.777857,0.992750,0.684619,0.663396,1.000000,0.739906,0.984824,0.745644,0.987940,0.734138,0.995700
5,0.575543,0.814985,0.989977,0.994181,0.739906,1.000000,0.779209,0.551708,0.835135,0.587275,0.799033
6,0.863370,0.986642,0.728084,0.709899,0.984824,0.779209,1.000000,0.734326,0.984546,0.727905,0.987526
7,0.580006,0.748255,0.591621,0.494659,0.745644,0.551708,0.734326,1.000000,0.736654,0.997742,0.742439
8,0.768479,0.999295,0.787625,0.771225,0.987940,0.835135,0.984546,0.736654,1.000000,0.735432,0.998035
9,0.571056,0.744783,0.631991,0.536048,0.734138,0.587275,0.727905,0.997742,0.735432,1.000000,0.737053


0.7765275490028436

In [24]:
hgmeans_tor_groups = [[] for i in range(5)]
kmeans_solutions=[]

import sys
import numpy as np
import random
import time
from sklearn.cluster import KMeans

MAX_FLOAT = sys.float_info.max


    
class Solution:
    #Private
    
    def __delete_assignment(self):
        self.__assignment = None
        
    #Public
               
    def __init__(self,assignment,cost = 0.0,alpha = 0.0):     
        self.__assignment = assignment 
        self.__cost = cost
        self.__alpha = alpha
        
    
    def getCost(self): 
        return self.__cost
    
    def getCostFromScratch(self, dataset, m):
        cost = 0
        centroid = self.assignmentToCentroid(dataset, m)
        for i in range(0, dataset.n):
                cost = cost + self.pcDist(i, centroid[self.__assignment[i]], dataset)
        return cost 

    def getAlpha(self):
        return self.__alpha
    
    def pcDist(self, p, center, dataset):
        total = 0
        for i in range(0,dataset.d):
            diff = dataset.data[p*(dataset.d)+i] - center[i]
            total = total + diff*diff
        return total
    
    def assignmentToCentroid(self, dataset, m):
        d = dataset.d 
        sizes = [None] * m
        centroid = [None] * m 
        
        for i in range(0,m):
            centroid[i] = [None] * d
            for j in range(0,d):
                centroid[i][j] = 0.0
            sizes[i] = 0
         
        for i in range(0,dataset.n):
            sizes[self.__assignment[i]] = sizes[self.__assignment[i]] + 1
            for j in range(0,dataset.d):
                centroid[self.__assignment[i]][j] = centroid[self.__assignment[i]][j] + dataset.data[i*d+j]
                
        for i in range(0,m):
            for j in range(0,d):
                if (sizes[i] != 0):
                    centroid[i][j] = (1.0*centroid[i][j])/sizes[i]
        sizes = None
        return centroid
    
    
    
    def centroidsToAssignment(self, centroid, dataset, m): 
        assignment = [None] * dataset.n
        for i in range(0, dataset.n):
            mindist = MAX_FLOAT
            for j in range(0,m):
                dist = self.pcDist(i, centroid[j], dataset)
                if np.any(dist < mindist):
                    mindist = dist
                    assignment[i] = j
        return assignment
    
    
    
class PbRun:

    
    def __init__(self, solution = [], time = 0.0, lastImprovment = 0, nbIter = 0.0, avgAlpha = 0.0, isValid = False):
        
        self.__solution = solution
        self.__time = time
        self.__lastImprovment = lastImprovment
        self.__nbIter = nbIter
        self.__avgAlpha = avgAlpha
        self.__isValid = isValid
        
    def getSolution(self):
        return self.__solution
    
    def getTime(self):
        return self.__time
    
    def getLastImprovment(self):
        return self.__lastImprovment
    
    def getNbIter(self):
        return self.nbIte
    
    def getAvgAlpha(self):
        return self.__avgAlpha
    
    def getIsValid(self):
        return self.__isValid
    
class Dataset:
    
    def __init__(self, dimensions, data, points):
        self.d = dimensions
        self.data = data
        self.n = points  

    
    
def readData():
        data = np.loadtxt("fisher.txt",skiprows=1)
        print(data)
        
        with open("fisher.txt") as f:
            first_line = f.readline()
        points, dimensions = map(int, first_line.split(" "))
        vetorized_data = [None] * (points * dimensions)
        
        for i in range(0,points):
            for j in range(0,dimensions):
                vetorized_data[i*dimensions+j] = data[i][j]
                
        print(vetorized_data)
        
def generateRandom(points,dimensions):
    
    data = [None] * points * dimensions
    for i in range(0,points):
        for j in range(0, dimensions):
            data[i*dimensions+j] = random.randint(0,1000)
    return data
        
        
def main(tor_data):
    clusters = 5
    points = random.randint(20,500)
    dimensions = random.randint(1,20)
    #data = generateRandom(points,dimensions)
    #print(data)
    print(points, dimensions, clusters)
    
    split_data = tor_data
    print(split_data)
    #dataset = Dataset(dimensions, data, points)
    
    start_time = time.time()
    kmeans_results = KMeans(n_clusters = clusters, random_state=0).fit(split_data)
    total_time = time.time() - start_time
    
    kmeans_solution = kmeans_results.labels_
    #kmeans_solutions = kmeans_solution
    solution = Solution(kmeans_solution,0,0)
    kmeans_clusters = kmeans_results.cluster_centers_
    kmeans_score = kmeans_results.score(split_data)
    pbRun = PbRun(solution,total_time)
    #print(kmeans_score)
    #print(solution.getCostFromScratch(dataset,clusters))
    print(total_time)
    print(kmeans_clusters)
    print(kmeans_solution)
    
    for i in range(5):
        for j in range(100):
            if(kmeans_solution[j]==i):
                hgmeans_tor_groups[i].append(f1[j])
    
    
main(tor_data)

31 17 5
[[0.00000000e+00 0.00000000e+00 3.84615385e-01 6.15384615e-01
  0.00000000e+00 0.00000000e+00]
 [3.99555226e-01 1.48257969e-03 1.30467013e-01 2.08302446e-01
  0.00000000e+00 2.60192735e-01]
 [0.00000000e+00 5.42944785e-01 0.00000000e+00 8.58895706e-02
  0.00000000e+00 3.71165644e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.06985525e-02 0.00000000e+00 1.06356199e-01 0.00000000e+00
  8.82945249e-01 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.16753927e-01 0.00000000e+00 1.83246073e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.05761317e-02 0.00000000e+00 9.76680384e-01 2.74348422e-03
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.60000000e-01 1.20000000e-01 4.00000000e-02
  0.00000000e+00 8.00000000e-02]
 [0.00000000e+00 0.00000000e+00 9.92537313

In [25]:
hgmeans_tor_groups

[['99436246@N00',
  '89613559@N07',
  '16326397@N02',
  '70554294@N00',
  '65438265@N00',
  '44359705@N07',
  '93241698@N00',
  '60142746@N00',
  '91008793@N00',
  '27168489@N00',
  '15205252@N00',
  '44654079@N05',
  '53874606@N00',
  '51035729697@N01',
  '62486370@N00'],
 ['51035681908@N01',
  '16926694@N06',
  '96684313@N00',
  '99127884@N00',
  '32044903@N04',
  '34961066@N00',
  '20075574@N00',
  '82845843@N04',
  '70289230@N00',
  '99771506@N00',
  '24395354@N02',
  '47554402@N00',
  '85801042@N00',
  '73416633@N00',
  '33906853@N00',
  '27947906@N08',
  '23339848@N00',
  '28157992@N03',
  '26344918@N04',
  '71206023@N00',
  '36357702@N08'],
 ['94725783@N00',
  '14878709@N00',
  '88276719@N00',
  '22996605@N05',
  '10502709@N05',
  '57519914@N00',
  '13273343@N00',
  '62373848@N00',
  '98185064@N00',
  '41894165897@N01',
  '26195569@N00',
  '9449875@N03',
  '23908938@N02',
  '91032493@N00',
  '61609579@N00',
  '27895744@N00',
  '26240579@N05',
  '43919105@N02',
  '24128704@N08',


In [26]:
hg_cosinT01_df01 = pd.DataFrame(columns=['Amusement','Beach','Cultural','Shopping','Sport','Structure'])
hg_tor_total=0
jac_tor_total=0
sum_com_top_int=0
for k in range(len(hgmeans_tor_groups)):

    cosT0 = pd.DataFrame(columns=['userID','poiTheme','sum'])

    for u in hgmeans_tor_groups[k]:
            cosT0=cosT0.append(tor_Interest_pref2.loc[(tor_Interest_pref2['userID']==u)])

    cosT0

    cosiT0 = pd.DataFrame({"tourist":cosT0['userID'],
                       "theme":cosT0['poiTheme'],
                       "Int_values":cosT0['sum']})

    cosinT0 = cosiT0.pivot_table(index='tourist',columns='theme',values='Int_values').fillna(0)
    cosinT0

    dict_com = cosinT0.to_dict()
    dict_com
    
    # for group interest preference
    cosinT01_df = cosinT0.mean(axis = 0, skipna = True)
    hg_cosinT01_df01 = hg_cosinT01_df01.append({'Amusement':cosinT01_df['Amusement'],'Beach':cosinT01_df['Beach'],'Cultural':cosinT01_df['Cultural'],'Shopping':cosinT01_df['Shopping'],'Sport':cosinT01_df['Sport'],'Structure':cosinT01_df['Structure']},ignore_index=True) 
    
    # Jaccard Similarity
    jac_similarityT0 = 1-pairwise_distances(cosinT0, metric="hamming")
    jac_simT0=pd.DataFrame(jac_similarityT0)
    jac_simT0

    jac_summT0 = 0
    jac_count=0
    for i in range(len(hgmeans_tor_groups[k])-1):
        for j in range(i+1,len(hgmeans_tor_groups[k])):
            jac_count+=1
            jac_summT0 = jac_summT0 + jac_simT0[i][j]
        
    jac_summT0 = jac_summT0/jac_count
    jac_summT0
    
    jac_tor_total=jac_tor_total+jac_summT0
    
    # Cosine Similarity
    similarityT0 = 1-pairwise_distances(cosinT0, metric="cosine")
    cos_simT0=pd.DataFrame(similarityT0)
    cos_simT0

    summT0 = 0
    count=0
    for i in range(len(hgmeans_tor_groups[k])-1):
        for j in range(i+1,len(hgmeans_tor_groups[k])):
            count+=1
            summT0 = summT0 + cos_simT0[i][j]
        
    summT0 = summT0/count
    summT0
    
    hg_tor_total=hg_tor_total+summT0
    
    # Common top interest
    cat_list=['Amusement','Beach','Cultural','Shopping','Sport','Structure']
    com_top_int = 0
    com_list = []
    for i in cat_list:
        com_sum = 0
        for j in hgmeans_tor_groups[k]:
            com_sum = com_sum + dict_com[i][j]
        com_sum = com_sum / len(hgmeans_tor_groups[k])
        com_list.append(com_sum)
    com_top_int = max(com_list)
    sum_com_top_int += com_top_int
    
sum_com_top_int=sum_com_top_int/len(hgmeans_tor_groups)    
jac_tor_total=jac_tor_total/len(hgmeans_tor_groups)    
hg_tor_total=hg_tor_total/len(hgmeans_tor_groups)
print("--------Result of hg-means algorithm--------\n")
print("jaccard similarity: ",jac_tor_total)
print("cosine similarity: ",hg_tor_total)
print("common top interest: ",sum_com_top_int)
print("\n")
print(hg_cosinT01_df01, sep='\n')

--------Result of hg-means algorithm--------

jaccard similarity:  0.3624135362845039
cosine similarity:  0.767208239109536
common top interest:  0.6838562843738599


   Amusement     Beach  Cultural  Shopping     Sport  Structure
0   0.793805  0.032993  0.124760  0.016667  0.008240   0.023535
1   0.011787  0.039322  0.887340  0.027187  0.017027   0.017337
2   0.110212  0.024730  0.131078  0.219781  0.242531   0.045863
3   0.048961  0.696335  0.031786  0.064846  0.064823   0.093249
4   0.015870  0.086901  0.058792  0.037037  0.002131   0.799269


In [27]:
#fcfs
pd.DataFrame(tor_Dur_min.groupby(["min","seqID","poiID"])["userID"].apply(list))

userID
min  seqID poiID                
0    17    7      [10031363@N00]
     83    13     [10575698@N02]
     104   30     [10627620@N06]
     510   16     [16048448@N00]
     519   30     [16478858@N00]
...                          ...
1438 3617  11     [53493629@N00]
1439 953   23     [20741443@N00]
     1255  25     [24930529@N07]
     1378  8      [27895744@N00]
     3367  16     [49109678@N04]

[7607 rows x 1 columns]

In [28]:
# for getting poi to their categories

tor_Interest2.groupby(['poiID','poiTheme']).apply(list)

poiID  poiTheme 
1      Sport        [userID, seqID, poiID, poiTheme, min, max, dur...
2      Sport        [userID, seqID, poiID, poiTheme, min, max, dur...
3      Sport        [userID, seqID, poiID, poiTheme, min, max, dur...
4      Sport        [userID, seqID, poiID, poiTheme, min, max, dur...
6      Cultural     [userID, seqID, poiID, poiTheme, min, max, dur...
7      Cultural     [userID, seqID, poiID, poiTheme, min, max, dur...
8      Cultural     [userID, seqID, poiID, poiTheme, min, max, dur...
9      Cultural     [userID, seqID, poiID, poiTheme, min, max, dur...
10     Cultural     [userID, seqID, poiID, poiTheme, min, max, dur...
11     Cultural     [userID, seqID, poiID, poiTheme, min, max, dur...
12     Cultural     [userID, seqID, poiID, poiTheme, min, max, dur...
13     Cultural     [userID, seqID, poiID, poiTheme, min, max, dur...
14     Amusement    [userID, seqID, poiID, poiTheme, min, max, dur...
15     Amusement    [userID, seqID, poiID, poiTheme, min, max, dur...
16 

In [29]:
dict_poiTotheme = {}

for i in range(1,6):
    dict_poiTotheme[i]="Sport"
for i in range(6,14):
    dict_poiTotheme[i]="Cultural"
for i in range(14,19):
    dict_poiTotheme[i]="Amusement"
for i in range(19,23):
    dict_poiTotheme[i]="Beach"
for i in range(23,28):
    dict_poiTotheme[i]="Shopping"
for i in range(28,31):
    dict_poiTotheme[i]="Structure"
dict_poiTotheme   

{1: 'Sport',
 2: 'Sport',
 3: 'Sport',
 4: 'Sport',
 5: 'Sport',
 6: 'Cultural',
 7: 'Cultural',
 8: 'Cultural',
 9: 'Cultural',
 10: 'Cultural',
 11: 'Cultural',
 12: 'Cultural',
 13: 'Cultural',
 14: 'Amusement',
 15: 'Amusement',
 16: 'Amusement',
 17: 'Amusement',
 18: 'Amusement',
 19: 'Beach',
 20: 'Beach',
 21: 'Beach',
 22: 'Beach',
 23: 'Shopping',
 24: 'Shopping',
 25: 'Shopping',
 26: 'Shopping',
 27: 'Shopping',
 28: 'Structure',
 29: 'Structure',
 30: 'Structure'}

In [30]:
dict_poiTotheme[1]

'Sport'

In [31]:
tor_costProf = pd.read_csv('costProfCat-ToroPOI-all.csv',delimiter=";")
tor_costProf

,from,to,cost,profit,category
0,1,2,3387.517345,609,Sport
1,1,3,2104.724858,688,Sport
2,1,4,834.021805,3056,Sport
3,1,6,1607.389271,986,Cultural
4,1,7,434.254844,2064,Cultural
...,...,...,...,...,...
807,30,25,1934.517567,1701,Shopping
808,30,26,10661.297058,104,Shopping
809,30,27,2909.802234,631,Shopping
810,30,28,808.684529,936,Structure


# Step2: POI Recommendation to Tour Group (POI2Group) 

In [32]:
from pulp import *

In [33]:
# dictionary of group interest preferences

group_intpref_values = cosinT01_df01.to_dict(orient='records')
group_intpref_values[1]

{'Amusement': 0.040719367290062106,
 'Beach': 0.5074400737655049,
 'Cultural': 0.031577306542677736,
 'Shopping': 0.03525109004264474,
 'Sport': 0.028749115650295295,
 'Structure': 0.3562630467088152}

In [34]:
poi_i = tor_costProf['from'].unique()
poi_j = tor_costProf['to'].unique()

poi_j = np.sort(poi_j)

poiId = tor_poi_pop['poiID'].values
N = len(poiId)
print(N)

a = [(i,j) for i in poi_i for j in poi_j]
len(a)

for i,j in a:
    if(i==j):
        a.remove((i,j))
        
print(len(a))
print(a)

30
812
[(1, 2), (1, 3), (1, 4), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (2, 1), (2, 3), (2, 4), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23), (2, 24), (2, 25), (2, 26), (2, 27), (2, 28), (2, 29), (2, 30), (3, 1), (3, 2), (3, 4), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (3, 17), (3, 18), (3, 19), (3, 20), (3, 21), (3, 22), (3, 23), (3, 24), (3, 25), (3, 26), (3, 27), (3, 28), (3, 29), (3, 30), (4, 1), (4, 2), (4, 3), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (4, 14), (4, 15), (4, 16), (4, 17), (4, 18), (4, 19), (4, 20), (4, 21), (4, 22), (4, 23), (4, 24), (4, 25), (4, 26), (4, 27), (4, 28), (4, 29), (4, 30), (6, 1), (6, 

In [35]:
poi_k=[]
for i in range(2,5):
    poi_k.append(i)
    
for i in range(6,30):
    poi_k.append(i)

# Cost between poi's

tor_costProf['new_col'] = a

tor_costProf

dict_cost = dict(zip(tor_costProf['new_col'],tor_costProf['cost']))
dict_cost

for i in range(1,31):
    if(i!=5):
        dict_cost[(i,5)]=1000000
        dict_cost[(5,i)]=1000000

    
print(dict_cost)

{(1, 2): 3387.51734497409, (1, 3): 2104.72485815982, (1, 4): 834.021804841558, (1, 6): 1607.38927095096, (1, 7): 434.254844101866, (1, 8): 562.232125979628, (1, 9): 8801.36705899733, (1, 10): 3013.46821209837, (1, 11): 2949.3723103730104, (1, 12): 25249.861158159103, (1, 13): 4576.84541583293, (1, 14): 3219.67992813099, (1, 15): 2600.6883113244603, (1, 16): 643.96381712263, (1, 17): 3199.36229225044, (1, 18): 5800.69655574051, (1, 19): 2505.0949107661604, (1, 20): 6763.69470382668, (1, 21): 1425.4697730906198, (1, 22): 1081.63791232899, (1, 23): 1189.3815874450802, (1, 24): 587.192559665223, (1, 25): 2150.28141013174, (1, 26): 10904.370489093, (1, 27): 3150.41084634237, (1, 28): 1040.4730641400301, (1, 29): 2355.54539482329, (1, 30): 244.250513958089, (2, 1): 3387.51734497409, (2, 3): 4502.56192376373, (2, 4): 2558.164379493, (2, 6): 3138.7724399715303, (2, 7): 3687.54008970777, (2, 8): 2836.7579806239696, (2, 9): 11357.3097781644, (2, 10): 5993.2470399204, (2, 11): 4337.46559564204, (

In [36]:
b=[(i,j) for i in range(1,N+1) for j in range(2,N+1)]
for i,j in b:
    if(i==j):
        b.remove((i,j))
        
for i in range(2,N):
    b.remove((30,i))
    
print(len(b))
print(b)

813
[(1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23), (2, 24), (2, 25), (2, 26), (2, 27), (2, 28), (2, 29), (2, 30), (3, 2), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (3, 17), (3, 18), (3, 19), (3, 20), (3, 21), (3, 22), (3, 23), (3, 24), (3, 25), (3, 26), (3, 27), (3, 28), (3, 29), (3, 30), (4, 2), (4, 3), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (4, 14), (4, 15), (4, 16), (4, 17), (4, 18), (4, 19), (4, 20), (4, 21), (4, 22), (4, 23), (4, 24), (4, 25), (4, 26), (4, 27), (4, 28), (4, 29), (4, 30), (5, 2),

In [37]:
# for group 0

group_to_poi={0:[],1:[],2:[],3:[],4:[]}
for g in range(5):

    dict_int = group_intpref_values[g]
    dict_cat = dict_poiTotheme
    dict_pop = dict(zip(tor_poi_pop['poiID'],tor_poi_pop['count']))

    eta=0.5

    # PROBLEM VARIABLE
    prob = LpProblem("MaxValue", LpMaximize)

    # DECISION VARIABLE
    r = LpVariable.dicts("poi_selected", 
                     [(i,j) for i,j in b],
                      0, 1, LpBinary)

    p = LpVariable.dicts("order_poi",
                     [i for i in range(2,N+1)],
                      2, N, LpInteger)

    # OBJECTIVE fUNCTION
    prob += lpSum( r[(i,j)] * ( dict_int[dict_cat[i]] * eta + dict_pop[i] * (1 - eta)) for i,j in b)

    # CONSTRAINTS
    prob += lpSum( (r[(1,j)] for j in range(2,N+1) ) ) == 1 

    prob += lpSum( (r[(i,N)] for i in range(1,N) ) ) == 1 

    prob += lpSum( (r[(1,j)] for j in range(2,N+1) ) ) == lpSum( (r[(i,N)] for i in range(1,N) ) )

    for k in range(2,N):
        prob += lpSum( (r[(i,k)] for i in range(1,N) if i!=k) ) <= 1 
        prob += lpSum( (r[(k,j)] for j in range(2,N+1) if k!=j) ) <= 1 
        prob += lpSum( (r[(i,k)] for i in range(1,N) if i!=k) ) == lpSum( (r[(k,j)] for j in range(2,N+1) if k!=j) )

    
    prob += lpSum( dict_cost[(i,j)] * r[(i,j)] for i,j in b ) <= 3500

    prob += r[(1,N)]==0

    for i in range(2,N):
        for j in range(2,N+1):
            if(i!=j):
                prob += p[i]-p[j]+1 <= (N-1)*(1-r[(i,j)])

    #SOLVE
    prob.solve()
    print("Status : ",LpStatus[prob.status])

    # PRINT OPTIMAL SOLUTION
    print("The Max Value = ",value(prob.objective))

    # Show which pois selected
    r_soln = np.array([r[(i,j)].varValue for i,j in b])
    p_soln = np.array([p[i].varValue for i in range(2,N+1)])
    print("poi_vars: ")
    print(r_soln)
    print("order: ")
    print(p_soln)
    
    for i,j in b:
        if(r[(i,j)].varValue==1):
            print((i,j))
            group_to_poi[g].append(i)
            group_to_poi[g].append(j)
            
    print("\n")

Status :  Optimal
The Max Value =  9222.424270594529
poi_vars: 
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

Status :  Optimal
The Max Value =  9222.428946267275
poi_vars: 
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [38]:
group_to_poi

{0: [1, 7, 4, 16, 7, 28, 8, 30, 16, 8, 22, 4, 28, 22],
 1: [1, 7, 4, 16, 7, 28, 8, 30, 16, 8, 22, 4, 28, 22],
 2: [1, 8, 4, 16, 7, 30, 8, 4, 16, 22, 22, 28, 28, 7],
 3: [1, 8, 4, 16, 7, 30, 8, 4, 16, 22, 22, 28, 28, 7],
 4: [1, 7, 4, 8, 7, 28, 8, 30, 16, 4, 22, 16, 28, 22]}

In [39]:
from collections import OrderedDict

In [40]:
list_IG0 = list(OrderedDict.fromkeys(group_to_poi[0]))
list_IG1 = list(OrderedDict.fromkeys(group_to_poi[1]))
list_IG2 = list(OrderedDict.fromkeys(group_to_poi[2]))
list_IG3 = list(OrderedDict.fromkeys(group_to_poi[3]))
list_IG4 = list(OrderedDict.fromkeys(group_to_poi[4]))
print(list_IG0)
print(list_IG1)
print(list_IG2)
print(list_IG3)
print(list_IG4)

[1, 7, 4, 16, 28, 8, 30, 22]
[1, 7, 4, 16, 28, 8, 30, 22]
[1, 8, 4, 16, 7, 30, 22, 28]
[1, 8, 4, 16, 7, 30, 22, 28]
[1, 7, 4, 8, 28, 30, 16, 22]


In [41]:
Ig=[]
cost_sum=0
for i,j in b:
    if(r[(i,j)].varValue==1):
        print((i,j),)
        cost_sum+=dict_cost[(i,j)]
        Ig.append(i)
        Ig.append(j)
        
print(cost_sum)

(1, 7)
(4, 8)
(7, 28)
(8, 30)
(16, 4)
(22, 16)
(28, 22)
3472.794018086659


In [42]:
tourist_to_cat = random_tor_user.set_index('tourist')
tourist_to_cat

theme,level_0,index,Amusement,Beach,Cultural,Shopping,Sport,Structure
tourist,,,,,,,,
94725783@N00,183,1335,0.000000,0.000000,0.384615,0.615385,0.000000,0.000000
14878709@N00,12,110,0.399555,0.001483,0.130467,0.208302,0.000000,0.260193
51035555243@N01,116,768,0.000000,0.542945,0.000000,0.085890,0.000000,0.371166
88276719@N00,170,1267,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
22996605@N05,28,228,0.010699,0.000000,0.106356,0.000000,0.882945,0.000000
...,...,...,...,...,...,...,...,...
55929357@N00,123,836,0.000000,0.441046,0.000000,0.000000,0.000000,0.558954
29619840@N00,55,382,0.353659,0.000000,0.402439,0.000000,0.243902,0.000000
32944866@N04,65,434,0.000000,0.992537,0.000000,0.000000,0.000000,0.007463


In [43]:
# tourist to their categories

tour_to_cat=tourist_to_cat.drop(columns=['index','level_0'])
tour_to_cat

theme,Amusement,Beach,Cultural,Shopping,Sport,Structure
tourist,,,,,,
94725783@N00,0.000000,0.000000,0.384615,0.615385,0.000000,0.000000
14878709@N00,0.399555,0.001483,0.130467,0.208302,0.000000,0.260193
51035555243@N01,0.000000,0.542945,0.000000,0.085890,0.000000,0.371166
88276719@N00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
22996605@N05,0.010699,0.000000,0.106356,0.000000,0.882945,0.000000
...,...,...,...,...,...,...
55929357@N00,0.000000,0.441046,0.000000,0.000000,0.000000,0.558954
29619840@N00,0.353659,0.000000,0.402439,0.000000,0.243902,0.000000
32944866@N04,0.000000,0.992537,0.000000,0.000000,0.000000,0.007463


In [45]:
dict_touristCat = tour_to_cat.to_dict('index')
dict_touristCat

{'94725783@N00': {'Amusement': 0.0,
  'Beach': 0.0,
  'Cultural': 0.38461538461538464,
  'Shopping': 0.6153846153846154,
  'Sport': 0.0,
  'Structure': 0.0},
 '14878709@N00': {'Amusement': 0.39955522609340255,
  'Beach': 0.0014825796886582653,
  'Cultural': 0.13046701260192736,
  'Shopping': 0.20830244625648628,
  'Sport': 0.0,
  'Structure': 0.2601927353595256},
 '51035555243@N01': {'Amusement': 0.0,
  'Beach': 0.5429447852760736,
  'Cultural': 0.0,
  'Shopping': 0.08588957055214724,
  'Sport': 0.0,
  'Structure': 0.37116564417177916},
 '88276719@N00': {'Amusement': 0.0,
  'Beach': 0.0,
  'Cultural': 0.0,
  'Shopping': 0.0,
  'Sport': 0.0,
  'Structure': 0.0},
 '22996605@N05': {'Amusement': 0.010698552548772814,
  'Beach': 0.0,
  'Cultural': 0.10635619886721209,
  'Shopping': 0.0,
  'Sport': 0.8829452485840151,
  'Structure': 0.0},
 '99436246@N00': {'Amusement': 1.0,
  'Beach': 0.0,
  'Cultural': 0.0,
  'Shopping': 0.0,
  'Sport': 0.0,
  'Structure': 0.0},
 '89613559@N07': {'Amusement

In [46]:
# Total Tour Interest of tourist t in a group 0

dict_tour_to_int={}
for i in tor_groups[0]:
    ToTi=0
    for j in list_IG0:
        ToTi += dict_touristCat[i][dict_cat[j]]
    dict_tour_to_int[i]=ToTi
    
dict_tour_to_int

{'22996605@N05': 1.9893014474512274,
 '62373848@N00': 1.5280814172634754,
 '98185064@N00': 1.9999999999999998,
 '41894165897@N01': 1.9999999999999998,
 '23908938@N02': 1.9203512072750082,
 '42907325@N00': 1.9920450778919458,
 '64834051@N00': 2.0,
 '35764233@N00': 1.977753058954394,
 '17373191@N00': 1.5531249999999999,
 '47501960@N00': 1.8384128809660722,
 '84518681@N00': 1.5419646217538576}

In [47]:
# POI2Group Evaluation

total_tour=0
count_tour=0
max_list=[]
min_list=[]
for i in dict_tour_to_int:
    count_tour+=1
    total_tour+=dict_tour_to_int[i]
    
total_tour=total_tour/count_tour
print("Tour total interest: ",total_tour)

for i in dict_tour_to_int:
    max_list.append(dict_tour_to_int[i])
    
print("Tour maximum interest: ",max(max_list))

for i in dict_tour_to_int:
    min_list.append(dict_tour_to_int[i])
    
print("Tour minimum interest: ",min(min_list))

Tour total interest:  1.8491849737778165
Tour maximum interest:  2.0
Tour minimum interest:  1.5280814172634754


# Step3: Tour Guide to Tour Group Assignment (Guide2Group)

In [48]:
#Using pyomo

from pyomo.environ import *
from coopr.pyomo import *

In [49]:
import random

# No of Groups
Groups = [0,1,2,3,4]

# poi's in groups
dict_groups_poi={}
for u in Groups:
    dict_groups_poi[u]={}
    
dict_groups_poi[0]=list_IG0
dict_groups_poi[1]=list_IG1
dict_groups_poi[2]=list_IG2
dict_groups_poi[3]=list_IG3
dict_groups_poi[4]=list_IG4

group_to_Poi = [(g,p) for g in Groups for p in dict_groups_poi[g]]
group_to_Poi

# Number of poi's
model_p_values = [1, 4, 7, 8, 16, 22, 28, 30]

# Model
model = ConcreteModel()

# Set for users or guides
model.u = Set(initialize=uniq_tor_users)

# Set for number of groups
model.g = Set(initialize=Groups)

# Binary variable for User-To-Group assignment
model.z = Var(model.u,model.g, within=Binary) # Z(u,g)

# Set for poi values 
model.p = Set(initialize=model_p_values)

# Binary variable for Group-To-Poi assignment
model.y = Var(model.g,model.p, within=Binary) # Y(g,p)


#Objective function
model.obj = Objective(expr = sum(((model.z[u,g]*model.y[g,p]*dict_guide_to_poi[p,u])) for g in model.g for u in model.u for p in model.p)
,sense=maximize)

#Constraints
def arule(model,u):
    return sum(model.z[u,g] for g in model.g) == 1
model.user_to_grp1 = Constraint(model.u,rule=arule)
    

model.pprint()

5 Set Declarations
    g : Dim=0, Dimen=1, Size=5, Domain=None, Ordered=False, Bounds=(0, 4)
        [0, 1, 2, 3, 4]
    p : Dim=0, Dimen=1, Size=8, Domain=None, Ordered=False, Bounds=(1, 30)
        [1, 4, 7, 8, 16, 22, 28, 30]
    u : Dim=0, Dimen=1, Size=196, Domain=None, Ordered=False, Bounds=None
        ['10502709@N05', '10627620@N06', '11090433@N05', '11191102@N07', '11241023@N03', '11349639@N00', '13273343@N00', '13528098@N00', '13644648@N03', '13672753@N00', '13907834@N00', '14391210@N00', '14878709@N00', '15205252@N00', '16048448@N00', '16326397@N02', '16693950@N00', '16926694@N06', '17373191@N00', '18412989@N00', '18948547@N00', '20075574@N00', '20342758@N00', '20344209@N00', '20456447@N03', '20741443@N00', '22111831@N05', '22979982@N06', '22996605@N05', '23119895@N00', '23269969@N00', '23339848@N00', '23908938@N02', '23987663@N00', '24128704@N08', '24395354@N02', '24616128@N00', '24854893@N00', '25133522@N00', '25475928@N04', '25561968@N00', '25802393@N02', '25846522@N00', 

        Key  : Active : Sense    : Expression
        None :   True : maximize : z[10502709@N05,0]*y[0,7]*0.2864101315148563 + z[10502709@N05,0]*y[0,22]*0.21759644740494036 + z[10502709@N05,0]*y[0,28]*1.4658119658119657 + z[10627620@N06,0]*y[0,30]*3.1859557867360206 + z[11090433@N05,0]*y[0,22]*0.4351928948098807 + z[11090433@N05,0]*y[0,30]*1.2743823146944082 + z[11191102@N07,0]*y[0,4]*0.06413612565445026 + z[11191102@N07,0]*y[0,7]*0.5728202630297126 + z[11191102@N07,0]*y[0,16]*0.2758232479594709 + z[11191102@N07,0]*y[0,22]*0.2719955592561754 + z[11191102@N07,0]*y[0,30]*0.12743823146944083 + z[11241023@N03,0]*y[0,7]*2.959571358986848 + z[11241023@N03,0]*y[0,22]*0.10879822370247018 + z[11241023@N03,0]*y[0,28]*1.0470085470085468 + z[11349639@N00,0]*y[0,4]*0.06413612565445026 + z[11349639@N00,0]*y[0,16]*0.055164649591894176 + z[11349639@N00,0]*y[0,30]*0.12743823146944083 + z[13273343@N00,0]*y[0,8]*0.11290322580645161 + z[13273343@N00,0]*y[0,16]*0.055164649591894176 + z[13273343@N00,0]*y[0,

        Key             : Lower : Body                                                                                                             : Upper : Active
           10502709@N05 :   1.0 :                z[10502709@N05,0] + z[10502709@N05,1] + z[10502709@N05,2] + z[10502709@N05,3] + z[10502709@N05,4] :   1.0 :   True
           10627620@N06 :   1.0 :                z[10627620@N06,0] + z[10627620@N06,1] + z[10627620@N06,2] + z[10627620@N06,3] + z[10627620@N06,4] :   1.0 :   True
           11090433@N05 :   1.0 :                z[11090433@N05,0] + z[11090433@N05,1] + z[11090433@N05,2] + z[11090433@N05,3] + z[11090433@N05,4] :   1.0 :   True
           11191102@N07 :   1.0 :                z[11191102@N07,0] + z[11191102@N07,1] + z[11191102@N07,2] + z[11191102@N07,3] + z[11191102@N07,4] :   1.0 :   True
           11241023@N03 :   1.0 :                z[11241023@N03,0] + z[11241023@N03,1] + z[11241023@N03,2] + z[11241023@N03,3] + z[11241023@N03,4] :   1.0 :   True
           11349

In [50]:
guideTogroup = []
results = SolverFactory("octeract-engine").solve(model,tee=True,keepfiles=False)
results.write() 
for v in model.component_data_objects(Var): 
    print(str(v), v.value)
    if(v.value==1):
        guideTogroup.append(str(v))
        
print(guideTogroup) # Guides assigned to the groups

Days left for current registered license: 73

 Octeract Engine v1.07.29    
 Copyright (c) Octeract Ltd, 2020

Loading problem...
Preprocessing problem...   5% complete

Preprocessor >>> Found solution during preprocessing.
Preprocessing problem... 100% complete
Presolve time : 2.83s


Objective value at global solution:  3.342e+02
Solution file written to: /tmp/tmphiq4k8l1.pyomo.octsol


Solved_To_Global_Optimality
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 196
  Number of variables: 1020
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Solved_To_Global_Optimality
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 4.163054943084717
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions di

z[35237095252@N01,1] 0.0
z[35237095252@N01,2] 0.0
z[35237095252@N01,3] 0.0
z[35237095252@N01,4] 0.0
z[35468159247@N01,0] 1.0
z[35468159247@N01,1] 0.0
z[35468159247@N01,2] 0.0
z[35468159247@N01,3] 0.0
z[35468159247@N01,4] 0.0
z[35764233@N00,0] 1.0
z[35764233@N00,1] 0.0
z[35764233@N00,2] 0.0
z[35764233@N00,3] 0.0
z[35764233@N00,4] 0.0
z[36357702@N08,0] 0.0
z[36357702@N08,1] 1.0
z[36357702@N08,2] 0.0
z[36357702@N08,3] 0.0
z[36357702@N08,4] 0.0
z[36553196@N08,0] 0.0
z[36553196@N08,1] 0.0
z[36553196@N08,2] 0.0
z[36553196@N08,3] 1.0
z[36553196@N08,4] 0.0
z[37517876@N07,0] 0.0
z[37517876@N07,1] 0.0
z[37517876@N07,2] 1.0
z[37517876@N07,3] 0.0
z[37517876@N07,4] 0.0
z[37663331@N00,0] 0.0
z[37663331@N00,1] 0.0
z[37663331@N00,2] 0.0
z[37663331@N00,3] 0.0
z[37663331@N00,4] 1.0
z[37804979@N00,0] 0.0
z[37804979@N00,1] 0.0
z[37804979@N00,2] 0.0
z[37804979@N00,3] 0.0
z[37804979@N00,4] 1.0
z[37996606796@N01,0] 0.0
z[37996606796@N01,1] 0.0
z[37996606796@N01,2] 0.0
z[37996606796@N01,3] 1.0
z[37996606796@N

z[84518681@N00,0] 0.0
z[84518681@N00,1] 0.0
z[84518681@N00,2] 0.0
z[84518681@N00,3] 0.0
z[84518681@N00,4] 1.0
z[84711541@N00,0] 0.0
z[84711541@N00,1] 0.0
z[84711541@N00,2] 1.0
z[84711541@N00,3] 0.0
z[84711541@N00,4] 0.0
z[84800532@N02,0] 0.0
z[84800532@N02,1] 0.0
z[84800532@N02,2] 1.0
z[84800532@N02,3] 0.0
z[84800532@N02,4] 0.0
z[84987970@N00,0] 0.0
z[84987970@N00,1] 0.0
z[84987970@N00,2] 1.0
z[84987970@N00,3] 0.0
z[84987970@N00,4] 0.0
z[85801042@N00,0] 0.0
z[85801042@N00,1] 1.0
z[85801042@N00,2] 0.0
z[85801042@N00,3] 0.0
z[85801042@N00,4] 0.0
z[86292040@N00,0] 0.0
z[86292040@N00,1] 0.0
z[86292040@N00,2] 0.0
z[86292040@N00,3] 1.0
z[86292040@N00,4] 0.0
z[87473264@N00,0] 0.0
z[87473264@N00,1] 0.0
z[87473264@N00,2] 0.0
z[87473264@N00,3] 1.0
z[87473264@N00,4] 0.0
z[88276719@N00,0] 1.0
z[88276719@N00,1] 0.0
z[88276719@N00,2] 0.0
z[88276719@N00,3] 0.0
z[88276719@N00,4] 0.0
z[89613559@N07,0] 0.0
z[89613559@N07,1] 0.0
z[89613559@N07,2] 0.0
z[89613559@N07,3] 1.0
z[89613559@N07,4] 0.0
z[9025385@

In [52]:
# Guide Total Expertise

summ_tot=0
for k in range(5):
    for i in tor_groups[k]:
        for j in list_IG0:
            summ_tot = summ_tot + dict_poi_count[i][j]
        
summ_tot

148.0853747869437

In [ ]:

    # PROBLEM VARIABLE
    prob1 = LpProblem("MaxValue", LpMaximize)

    # DECISION VARIABLE
    r = LpVariable.dicts("poi_selected", 
                     [(i,j) for i,j in b],
                      0, 1, LpBinary)

    p = LpVariable.dicts("order_poi",
                     [i for i in range(2,N+1)],
                      2, N, LpInteger)

    # OBJECTIVE fUNCTION
    prob += lpSum( r[(i,j)] * ( dict_int[dict_cat[i]] * eta + dict_pop[i] * (1 - eta)) for i,j in b)

    # CONSTRAINTS
    prob += lpSum( (r[(1,j)] for j in range(2,N+1) ) ) == 1 

    prob += lpSum( (r[(i,N)] for i in range(1,N) ) ) == 1 

    prob += lpSum( (r[(1,j)] for j in range(2,N+1) ) ) == lpSum( (r[(i,N)] for i in range(1,N) ) )

    for k in range(2,N):
        prob += lpSum( (r[(i,k)] for i in range(1,N) if i!=k) ) <= 1 
        prob += lpSum( (r[(k,j)] for j in range(2,N+1) if k!=j) ) <= 1 
        prob += lpSum( (r[(i,k)] for i in range(1,N) if i!=k) ) == lpSum( (r[(k,j)] for j in range(2,N+1) if k!=j) )

    
    prob += lpSum( dict_cost[(i,j)] * r[(i,j)] for i,j in b ) <= 3500

# Rough Work

In [50]:
# Cost Matrix (groups to guides)

G = {}
for u in range(196):
    G[u]={}

for k in Groups:
    for i in uniq_tor_users:
        total_expertise = 0
        for j in model_p_values:
            total_expertise += math.ceil(dict_guide_to_poi[j,i])
        G[k][i] = total_expertise
        
'''for i in range(5,196):
    for j in uniq_tor_users:
        G[i][j] =33'''

print(G)  

pd.DataFrame.from_dict(G,orient='index')

{0: {'10502709@N05': 4, '10627620@N06': 4, '11090433@N05': 3, '11191102@N07': 5, '11241023@N03': 6, '11349639@N00': 3, '13273343@N00': 3, '13528098@N00': 3, '13644648@N03': 9, '13672753@N00': 4, '13907834@N00': 8, '14391210@N00': 4, '14878709@N00': 10, '15205252@N00': 0, '16048448@N00': 6, '16326397@N02': 2, '16693950@N00': 8, '16926694@N06': 4, '17373191@N00': 4, '18412989@N00': 6, '18948547@N00': 2, '20075574@N00': 1, '20342758@N00': 5, '20344209@N00': 1, '20456447@N03': 13, '20741443@N00': 33, '22111831@N05': 2, '22979982@N06': 0, '22996605@N05': 4, '23119895@N00': 3, '23269969@N00': 3, '23339848@N00': 1, '23908938@N02': 18, '23987663@N00': 2, '24128704@N08': 2, '24395354@N02': 0, '24616128@N00': 2, '24854893@N00': 2, '25133522@N00': 1, '25475928@N04': 2, '25561968@N00': 3, '25802393@N02': 5, '25846522@N00': 3, '26195569@N00': 0, '26240579@N05': 3, '26344918@N04': 4, '26736723@N00': 3, '27008470@N00': 3, '27031977@N04': 2, '27168489@N00': 4, '27895744@N00': 3, '27947906@N08': 3, '28

,10502709@N05,10627620@N06,11090433@N05,11191102@N07,11241023@N03,11349639@N00,13273343@N00,13528098@N00,13644648@N03,13672753@N00,...,96684313@N00,97675182@N00,98185064@N00,98222757@N02,98715075@N00,9911655@N08,99127884@N00,99436246@N00,99771506@N00,9985167@N04
0,4,4,3,5,6,3,3,3,9,4,...,8,0,3,2,4,8,5,2,2,3
1,4,4,3,5,6,3,3,3,9,4,...,8,0,3,2,4,8,5,2,2,3
2,4,4,3,5,6,3,3,3,9,4,...,8,0,3,2,4,8,5,2,2,3
3,4,4,3,5,6,3,3,3,9,4,...,8,0,3,2,4,8,5,2,2,3
4,4,4,3,5,6,3,3,3,9,4,...,8,0,3,2,4,8,5,2,2,3


In [51]:
G = {
    '0' : {'#1': 4, '#2': 4, '#3': 3, '#4': 5, '#5': 6},
    '1' : {'#1': 4, '#2': 4, '#3': 3, '#4': 5, '#5': 6},
    '2' : {'#1': 4, '#2': 4, '#3': 3, '#4': 5, '#5': 6},
    '3' : {'#1': 33, '#2': 33, '#3': 33, '#4': 33, '#5': 33},
    '4' : {'#1': 33, '#2': 33, '#3': 33, '#4': 33, '#5': 33}
}

G

{'0': {'#1': 4, '#2': 4, '#3': 3, '#4': 5, '#5': 6},
 '1': {'#1': 4, '#2': 4, '#3': 3, '#4': 5, '#5': 6},
 '2': {'#1': 4, '#2': 4, '#3': 3, '#4': 5, '#5': 6},
 '3': {'#1': 33, '#2': 33, '#3': 33, '#4': 33, '#5': 33},
 '4': {'#1': 33, '#2': 33, '#3': 33, '#4': 33, '#5': 33}}

In [61]:
algorithm.find_matching(G, matching_type = 'min', return_type = 'list')

KeyboardInterrupt: 

In [52]:
from hungarian_algorithm import algorithm

In [ ]:
algorithm.find_matching(G, matching_type = 'max', return_type = 'list')

In [64]:
import ipopt

In [52]:
#Using Pulp

# No of Groups
G=[0,1,2,3,4]

# poi's in groups
dict_groups_poi={}
for u in G:
    dict_groups_poi[u]={}
    
dict_groups_poi[0]=list_IG0
dict_groups_poi[1]=list_IG1
dict_groups_poi[2]=list_IG2
dict_groups_poi[3]=list_IG3
dict_groups_poi[4]=list_IG4

# PROBLEM VARIABLE
prob1 = LpProblem("MaxValue", LpMaximize)

# Decision Variables
z = LpVariable.dicts("tour_guide_assign", 
                         [(u,g) for u in uniq_tor_users for g in G],
                          0, 1, LpBinary)

y = LpVariable.dicts("group_rec_poi", 
                         [(g,p) for g in G for p in dict_groups_poi[g]],
                          0, 1, LpBinary)

# Objective Function
prob1 += lpSum( z[(u,g)]*y[(g,p)]*(guide_to_poi[p][u]) for g in G for u in uniq_tor_users for p in dict_groups_poi[g])

# Constraints
prob1 += lpSum( z[(u,g)]) <= 1

#prob1 += lpSum( z[(g,u)]) <= 1

#SOLVE
prob.solve()
print("Status : ",LpStatus[prob.status])

# PRINT OPTIMAL SOLUTION
print("The Max Value = ",value(prob.objective))

# Show which tour guides selected
z_soln = np.array([z[(u,g)].varValue for u in uniq_tor_users for g in G])
y_soln = np.array([y[(g,p)].varValue for g in G for p in dict_groups_poi[g]])
print("tour_guides: ")
print(z_soln)
print("group_to_poi: ")
print(y_soln)

TypeError: Non-constant expressions cannot be multiplied